In [ ]:
#| default_exp machine_learning.tokenize.def_and_notat_token_classification

# machine_learning.tokenize.def_and_notat_token_classification
> Functions for gathering and processing tokenization data and for using ML models trained with such data.

Previous, `trouver` just had functionalities for using ML models to identify newly introduced notations in text and for gathering data to train such models. Moreover, such models were merely classification models, and using these models to identify newly introduced notations had a lot of computational redundancies.

This module aims to provide the same functionalities for both definitions and notations by training and using token classification models instead.

In [ ]:
# TODO: Create a new module dedicated to definition and notation identification and move approparite functions over there. 

In [ ]:
#| export
from collections.abc import Callable
import copy
from itertools import pairwise
import os 
from os import PathLike
from pathlib import Path
import random
from typing import Literal, Optional, TypedDict, Union
import warnings

import bs4
from openai import OpenAI
from pydantic import BaseModel
import regex
from transformers import BatchEncoding, pipelines, PreTrainedTokenizer, PreTrainedTokenizerFast

from trouver.helper import is_not_space_and_not_punc, split_string_at_indices
from trouver.helper.definition_and_notation import double_asterisk_indices, notation_asterisk_indices
from trouver.helper.html import (
    add_HTML_tag_data_to_raw_text, add_space_to_lt_symbols_without_space, remove_html_tags_in_text, StrAndHTMLTagsWithIndices,
    HTMLTagWithIndices)
from trouver.helper.latex.core import _is_balanced_braces, _first_curly_bracket, _last_curly_bracket
from trouver.helper.latex.augment import (
    augment_text, change_font_styles_at_random, change_greek_letters_at_random, dollar_sign_manipulation, push_dollar_sign, random_char_modification, random_latex_command_removal, random_word_removal, remove_font_styles_at_random, remove_math_keywords)

from trouver.helper.regex import latex_indices, replace_string_by_indices
from trouver.obsidian.file import MarkdownFile, MarkdownLineEnum
from trouver.personal_vault.note_processing import process_standard_information_note
from trouver.obsidian.vault import VaultNote




In [ ]:
from unittest import mock
import shutil
import tempfile

from datasets import ClassLabel, Dataset, Features, Sequence, Value
from transformers import AutoTokenizer
from fastcore.test import *

from trouver.helper.tests import _test_directory


## Parse data from information notes

In [ ]:
#| export
def convert_double_asterisks_to_html_tags(
        text: str
        ) -> str:
    """
    Replace the double asterisks, which signify definitions and notations,
    in `text` with HTML tags.
    """
    double_asts = double_asterisk_indices(text)
    replacement_html_tags = [
        _html_tag_from_double_ast(text[start:end])
        for start, end in double_asts]
    return replace_string_by_indices(
        text, double_asts, replacement_html_tags)


def _html_tag_from_double_ast(
        double_ast_string: str # Starts and ends with double asts
        ) -> str:
    """
    Get the HTML tag representing definition or notation data from
    a string surrounded by double asterisks.

    This is used in the `_convert_double_asterisks_to_html_tags` function.
    """
    no_asts = double_ast_string[2:-2]
    if notation_asterisk_indices(double_ast_string):
        return f'<span notation="">{no_asts}</span>'
    else:
        return f'<b definition="">{no_asts}</b>'


In [ ]:
print(convert_double_asterisks_to_html_tags("**hi**. Here is a notation **$asdf$**"))
test_eq(convert_double_asterisks_to_html_tags("**hi**. Here is a notation **$asdf$**"), '<b definition="">hi</b>. Here is a notation <span notation="">$asdf$</span>')


<b definition="">hi</b>. Here is a notation <span notation="">$asdf$</span>


In [ ]:
#| export
def raw_text_with_html_tags_from_markdownfile(
        mf: MarkdownFile,
        vault: PathLike
        ) -> str:
    """
    Process the `MarkdownFile`, replacing the double asterisk surrounded
    text indicating definitions and notations to be HTML tags instead.
    """
    mf = process_standard_information_note(
        mf, vault, remove_double_asterisks=False,
        remove_html_tags=False)
    return convert_double_asterisks_to_html_tags(str(mf))



In [ ]:
#| hide

# TODO: 
# I want to make sure that footnotes are getting properly removed.
mf = MarkdownFile.from_string(
    r"""---
aliases: []
tags: []
---
# Something  

Some kind of potato[^2]

[^2]: Some footnote

[[link_to_note|Some link]]


# See Also
# Meta
## References and Citations
""") 
raw_text_with_html_tags_from_markdownfile(mf, None)


'Some kind of potato[^2]\n\n[^2]: Some footnote\n\nSome link\n'

In [ ]:
#| hide
mf = MarkdownFile.from_string(
    r"""---
aliases: []
tags: []
---
# Galois group of a separable and normal finite field extension

Let $L/K$ be a separable and normal finite field extension. Its <b definition="Galois group of a separable and normal finite field extension">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span> is...

# Galois group of a separable and normal profinite field extension

In fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that
$L = \varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its **Galois group** **$\operatorname{Gal}(L/K)$**

# See Also
# Meta
## References and Citations
""")

In the following example, let `mf` be the following `MarkdownFile`:

In [ ]:
print(str(mf))

---
aliases: []
tags: []
---
# Galois group of a separable and normal finite field extension

Let $L/K$ be a separable and normal finite field extension. Its <b definition="Galois group of a separable and normal finite field extension">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span> is...

# Galois group of a separable and normal profinite field extension

In fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that
$L = \varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its **Galois group** **$\operatorname{Gal}(L/K)$**

# See Also
# Meta
## References and Citations


The `raw_text_with_html_tags_from_markdownfile` function processes the `MarkdownFile` much in the same way as the `process_standard_information_note` function, except it 1. preserves HTML tags, and 2. replaces text surrounded by double asterisks `**` with HTML tags signifiying whether the text displays a definition or a notation.

In the below example, note that the `vault` parameter is set to `None`; this is fine for this example becaues the `process_standard_information_note` function only needs a `vault` argument when embedded links need to be replaced with text (via the `MarkdownFile.replace_embedded_links_with_text` function), but `mf` has no embedded links.

In [ ]:
print(raw_text_with_html_tags_from_markdownfile(mf, None))

Let $L/K$ be a separable and normal finite field extension. Its <b definition="Galois group of a separable and normal finite field extension">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span> is...

In fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that
$L = \varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its <b definition="">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span>



In [ ]:
#| hide
assert '**' not in raw_text_with_html_tags_from_markdownfile(mf, None)

In [ ]:
#| export
class HTMLData(TypedDict):
    note_name: str
    raw_text: str
    tags: list[HTMLTagWithIndices]
    # list[bs4.element.Tag]


In [ ]:
#| export
def html_data_from_note(
        note_or_mf: Union[VaultNote, MarkdownFile], # Either a `VaultNote`` object to a note or a `MarkdownFile` object from which to extra html data.
        vault: Optional[PathLike] = None, # If vault to use when processing the `MarkdownFile` objects (if `note_of_mf` is a `VaultNote`, then this `MarkdownFile` object is created from the text of the note), cf. the `process_standard_information_note` function.
        note_name: Optional[str] = None, # If `note_or_mf` is a `MarkdownFile`, `note_name` should be the name of the note from which the `MarkdownFile` comes from if applicable. If `note_or_mf` is a `VaultNote` object, then `note_name` is ignored and `note_or_mf.name` is used instead.
        ) -> Union[HTMLData, None]: # The keys to the dict are "note_name", "raw_text", "tags". However, `None` is returned if `note` does not exist or the note is marked with auto-generated, unverified data.
    # TODO: implement obtaining multiple datapoints from a single note
    # Via typos for example.
    # TODO: implement various data augmentation techniques
    """Obtain html data for token classification from the information note.

    Currently, the token types mainly revolve around definitions and
    notations.

    If `note` has the tag `_auto/def_and_notat_identified`, then the data
    in the note is assumed to be auto-generated and not verified and
    `None` is returned.

    **Returns**
    - Union[dict, None]
        - The keys-value pairs are 
            - `"note_name"` - The name of the note
            - `"raw_text"` - The raw text to include in the data.
            - `"tags"` - The list with HTML tags carrying definition/notation
              data and their locations in the Raw text. See the second output to
              the function `remove_html_tags_in_text`.
                - Each element of the list is a tuple consisting of a ``bs4.element.Tag``
                  and two ints.
    """
    if isinstance(note_or_mf, VaultNote) and not note_or_mf.exists():
        return None
    if isinstance(note_or_mf, VaultNote):
        mf = MarkdownFile.from_vault_note(note_or_mf)
        note_name = note_or_mf.name
        if vault is None:
            vault = note_or_mf.vault
    else: # isinstance(note_or_mf, MarkdownFile):
        mf = note_or_mf.copy(deep=False)
    if mf.has_tag('_auto/def_and_notat_identified'):
        return None
    raw_text_with_tags = raw_text_with_html_tags_from_markdownfile(mf, vault)
    raw_text, tags_and_locations = remove_html_tags_in_text(raw_text_with_tags)

    return HTMLData(note_name=note_name, raw_text=raw_text, tags=tags_and_locations)

In the following example, we mock a `VaultNote` whose content is that of `mf` in the example for the `raw_text_with_html_tags_from_markdownfile` function. Note that there is some text surrounded by double within `mf` surrounded by double asterisks `**` and some text surrounded by HTML tags to indicate definitions and notations introduced.

In [ ]:
mf = MarkdownFile.from_string(
    r"""---
aliases: []
tags: []
---
# Galois group of a separable and normal finite field extension

Let $L/K$ be a separable and normal finite field extension. Its <b definition="Galois group of a separable and normal finite field extension">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span> is...

# Galois group of a separable and normal profinite field extension

In fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that
$L = \varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its **Galois group** **$\operatorname{Gal}(L/K)$**

# See Also
# Meta
## References and Citations
""")

with (mock.patch('__main__.VaultNote') as mock_VaultNote,
      mock.patch('__main__.MarkdownFile.from_vault_note') as mock_from_vault_note,
      mock.patch('__main__.isinstance') as mock_isinstance):
    mock_VaultNote.exists.return_value = True
    mock_VaultNote.name = "Note's name"
    mock_from_vault_note.return_value = mf
    mock_isinstance.return_value = True

    print(f"The following is the text from mf:\n\n{str(mf)}")

    html_data = html_data_from_note(mock_VaultNote, None)
    print(html_data)

    test_eq(html_data['note_name'], "Note's name")
    assert '**' not in html_data['raw_text']
    assert '<' not in html_data['raw_text']  # Test the lack of HTML tags in the raw text

    print(html_data['tags'])
    test_eq(len(html_data['tags']), 4)
    assert isinstance(html_data['tags'][0][0], bs4.element.Tag)
    assert html_data['tags'][0][0].has_attr('definition')
    assert not html_data['tags'][0][0].has_attr('notation')
    assert html_data['tags'][1][0].has_attr('notation')
    assert not html_data['tags'][1][0].has_attr('definition')
    assert html_data['tags'][2][0].has_attr('definition')
    assert not html_data['tags'][2][0].has_attr('notation')
    assert html_data['tags'][3][0].has_attr('notation')
    assert not html_data['tags'][3][0].has_attr('definition')

The following is the text from mf:

---
aliases: []
tags: []
---
# Galois group of a separable and normal finite field extension

Let $L/K$ be a separable and normal finite field extension. Its <b definition="Galois group of a separable and normal finite field extension">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span> is...

# Galois group of a separable and normal profinite field extension

In fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that
$L = \varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its **Galois group** **$\operatorname{Gal}(L/K)$**

# See Also
# Meta
## References and Citations
{'note_name': "Note's name", 'raw_text': 'Let $L/K$ be a separable and normal finite field extension. Its Galois group $\\operatorname{Gal}(L/K)$ is...\n\nIn fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal pr

We can also just pass a `MarkdonwFile` object instead of a `VaultNote` object. In this case, we can specify the `note_name` parameter to indicate which note the `MarkdownFile` object came from, if applicable.

In [ ]:
html_data = html_data_from_note(mf, vault=None, note_name="Note's name")
print(html_data)

test_eq(html_data['note_name'], "Note's name")
assert '**' not in html_data['raw_text']
assert '<' not in html_data['raw_text']  # Test the lack of HTML tags in the raw text

print(html_data['tags'])
test_eq(len(html_data['tags']), 4)
assert isinstance(html_data['tags'][0][0], bs4.element.Tag)
assert html_data['tags'][0][0].has_attr('definition')
assert not html_data['tags'][0][0].has_attr('notation')
assert html_data['tags'][1][0].has_attr('notation')
assert not html_data['tags'][1][0].has_attr('definition')
assert html_data['tags'][2][0].has_attr('definition')
assert not html_data['tags'][2][0].has_attr('notation')
assert html_data['tags'][3][0].has_attr('notation')
assert not html_data['tags'][3][0].has_attr('definition')

{'note_name': "Note's name", 'raw_text': 'Let $L/K$ be a separable and normal finite field extension. Its Galois group $\\operatorname{Gal}(L/K)$ is...\n\nIn fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that\n$L = \\varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its Galois group $\\operatorname{Gal}(L/K)$\n', 'tags': [HTMLTagWithIndices(tag=<b definition="Galois group of a separable and normal finite field extension">Galois group</b>, start=64, end=76), HTMLTagWithIndices(tag=<span notation="">$\operatorname{Gal}(L/K)$</span>, start=77, end=102), HTMLTagWithIndices(tag=<b definition="">Galois group</b>, start=330, end=342), HTMLTagWithIndices(tag=<span notation="">$\operatorname{Gal}(L/K)$</span>, start=343, end=368)]}
[HTMLTagWithIndices(tag=<b definition="Galois group of a separable and normal finite field extension">Galois group</b>, start=64, end=76), HTMLTagWithIndices(tag

If we do not specify `note_name`, then `None` is used for the `'Note name'` key in the output:

In [ ]:
html_data = html_data_from_note(mf, vault=None, note_name=None)
print(html_data)

assert html_data['note_name'] is None

{'note_name': None, 'raw_text': 'Let $L/K$ be a separable and normal finite field extension. Its Galois group $\\operatorname{Gal}(L/K)$ is...\n\nIn fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that\n$L = \\varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its Galois group $\\operatorname{Gal}(L/K)$\n', 'tags': [HTMLTagWithIndices(tag=<b definition="Galois group of a separable and normal finite field extension">Galois group</b>, start=64, end=76), HTMLTagWithIndices(tag=<span notation="">$\operatorname{Gal}(L/K)$</span>, start=77, end=102), HTMLTagWithIndices(tag=<b definition="">Galois group</b>, start=330, end=342), HTMLTagWithIndices(tag=<span notation="">$\operatorname{Gal}(L/K)$</span>, start=343, end=368)]}


For the following example, the note has an HTML tag already with extra data (attributes other than `'definition'` or `'notation'`). We assert that the extra data is preserved. 

In [ ]:
with (mock.patch('__main__.VaultNote') as mock_VaultNote,
      mock.patch('__main__.MarkdownFile.from_vault_note') as mock_from_vault_note,
      mock.patch('__main__.isinstance') as mock_isinstance):
    mock_VaultNote.exists.return_value = True
    mock_VaultNote.name = "Note's name"
    mock_isinstance.return_value = True

    text = r'Let $X$ be a topological space and let $U \subseteq X$ be an subspace. The <b definition="Closure of a subspace of a topological space" typo="dosure of $U$">closure of $U$</b> is defined as...'
    mf = MarkdownFile.from_string(text)
    mock_from_vault_note.return_value = mf
    print(f"The following is the text of the mocked note: \n\n {text}\n\n")

    html_data = html_data_from_note(mock_VaultNote, None)
    print(html_data)
    assert html_data['tags'][0][0].has_attr('typo')
    test_eq(html_data['tags'][0][0].attrs['typo'], 'dosure of $U$')

The following is the text of the mocked note: 

 Let $X$ be a topological space and let $U \subseteq X$ be an subspace. The <b definition="Closure of a subspace of a topological space" typo="dosure of $U$">closure of $U$</b> is defined as...


{'note_name': "Note's name", 'raw_text': 'Let $X$ be a topological space and let $U \\subseteq X$ be an subspace. The closure of $U$ is defined as...', 'tags': [HTMLTagWithIndices(tag=<b definition="Closure of a subspace of a topological space" typo="dosure of $U$">closure of $U$</b>, start=75, end=89)]}


In the following example, the (mocked) note has the `#_auto/def_and_notats_identified` tag to indicate that its definition and notation markings were auto-generated by a model (trained with data processed by the `tokenize_html_data` function) using the `auto_mark_def_and_notats` function. In this case, the `html_data_from_note` function returns `None` to prevent gathering data that is unverified and auto-generated by a model.

In [ ]:
# with (mock.patch('__main__.VaultNote') as mock_VaultNote,
#       mock.patch('__main__.MarkdownFile.from_vault_note') as mock_from_vault_note):
#     mock_VaultNote.exists.return_value = True
#     mock_VaultNote.name = "Note's name"
text = r'''---
tags: [_auto/def_and_notat_identified]
---
Let $X$ be a topological space and let $U \subseteq X$ be an subspace. The <b definition="Closure of a subspace of a topological space" typo="dosure of $U$">closure of $U$</b> is defined as...'''

mf = MarkdownFile.from_string(text)
mock_from_vault_note.return_value = mf
print(f"The following is the text of the mocked note: \n\n{text}\n\n")

html_data = html_data_from_note(note_or_mf=mf)
assert(html_data is None)

The following is the text of the mocked note: 

---
tags: [_auto/def_and_notat_identified]
---
Let $X$ be a topological space and let $U \subseteq X$ be an subspace. The <b definition="Closure of a subspace of a topological space" typo="dosure of $U$">closure of $U$</b> is defined as...




In [ ]:
#| export
def tokenize_html_data(
        html_locus: HTMLData, # An output of `html_data_from_note`
        tokenizer: Union[PreTrainedTokenizer, PreTrainedTokenizerFast],
        max_length: int, # Max length for each sequence of tokens
        ner_tag_from_html_tag: Callable[[bs4.element.Tag], Union[str, 'None']], # takes in a bs4.element.Tag and outputs the ner_tag (as a string or `None`)
        label2id: dict[str, int], # The keys are ner_tag's of the form f"I-{output}" or f"B-{output}" where `output` is an output of `ner_tag_from_html_tag`.
        default_label: str = "O", # The default label for the NER tagging.
        ) -> tuple[list[list[str]], list[list[int]]]: # The first list consists of the tokens and the second list consists of the named entity recognition tags.
    """Actually tokenize the html data outputted by `html_data_from_note`.

    To account for the possibility that the raw text is long,
    this function uses the `tokenizer.batch_encode_plus` function
    to tokenize the text into sequences. 
    """
    tokenized = tokenizer.batch_encode_plus(
        [html_locus["raw_text"]], max_length=max_length, return_overflowing_tokens=True,
        return_offsets_mapping=True, truncation=True)

    default_id = label2id[default_label]        
    ner_ids = [[default_id for _ in seq_input_ids]
               for seq_input_ids in tokenized['input_ids']]
    for tag, start, end in html_locus['tags']:
        ner_tag = ner_tag_from_html_tag(tag)
        if ner_tag is None:
            continue  # `ner_tag` is not of relevant data.
        tuppy = _start_end_seqs_indices_for_html_tag(tokenized, start, end - 1)
        (start_seq, start_index_in_seq), (end_seq, end_index_in_seq) = tuppy
        _set_ner_ids_for_tag(
            ner_ids, start_seq, start_index_in_seq, end_seq, end_index_in_seq,
            label2id, ner_tag)
    # return tokenized["input_ids"], ner_ids
    tokens = [tokenizer.convert_ids_to_tokens(tokens_for_seq)
              for tokens_for_seq in tokenized["input_ids"]]
    return tokens, ner_ids


def _start_end_seqs_indices_for_html_tag(
        tokenized: BatchEncoding,
        tag_start_ind: int,
        tag_end_ind: int
        ) -> tuple[tuple[int, int], tuple[int, int]]: # The first tuple is `(a, b)` where `tokenized['input_ids'][a][b]` is the token corresponding to the start of the HTML tag's (raw) text. The second tuple is `(c, d)` where `tokenized['input_ids'][c][d]` is the token corresponding to the end of the HTML tag's (raw) text.
    start_seq = _search_seq_ind_for_char(tokenized['offset_mapping'], tag_start_ind)
    # start_index_in_seq = tokenized.char_to_token(batch_or_char_index=start_seq, char_index=tag_start_ind)
    start_index_in_seq = _search_within_seq_for_char(tokenized['offset_mapping'][start_seq], tag_start_ind)
    end_seq = _search_seq_ind_for_char(tokenized['offset_mapping'], tag_end_ind)
    # end_index_in_seq = tokenized.char_to_token(batch_or_char_index=end_seq, char_index=tag_end_ind)
    end_index_in_seq = _search_within_seq_for_char(tokenized['offset_mapping'][end_seq], tag_end_ind)
    return (start_seq, start_index_in_seq), (end_seq, end_index_in_seq)


def _min_max_char_ind_for_seq(
        offset_for_seq: list[tuple[int,int]] # An item in tokenized['offset_mapping']
        ):
    min_char_ind, max_char_ind = 0, 0
    for inds in offset_for_seq:
        if inds != (0,0):
            min_char_ind = inds[0]
            break
    for inds in reversed(offset_for_seq):
        if inds != (0,0):
            max_char_ind = inds[1]
            break
    return min_char_ind, max_char_ind

def _char_is_in_seq(
        offset_for_seq: list[int], # An item in tokenized['offset_mapping']
        char: int # The index of a character in the original raw text
        ) -> bool:
    min_char_ind, max_char_ind = _min_max_char_ind_for_seq(offset_for_seq)
    return min_char_ind <= char and char < max_char_ind

def _search_seq_ind_for_char(
        offsets: list[tuple[int, int]], # tokenized['offset_mapping']
        char: int # The index of a character in the original raw text
        ) -> int:
    """
    Binary search the index of the sequence containing the token at the 
    location of the index `char` within the original (raw) text.

    Based on pseudocode from https://pseudoeditor.com/guides/binary-search
    """
    left = 0
    right = len(offsets) - 1
    while left <= right:
        mid = (left + right) // 2
        min_char_ind, max_char_ind = _min_max_char_ind_for_seq(offsets[mid])
        if min_char_ind <= char and char < max_char_ind:
            return mid
        elif max_char_ind <= char:
            left = mid + 1
        else:
            right = mid - 1
    return -1  # This should not be returned under normal use.


def _search_within_seq_for_char(
        seq_offset: list[tuple[int, int]],
        char: int
    ) -> int:
    """
    Binary search for the index within the sequence corresponding
    to the token at the location of the index `char` within the
    original (raw) text.

    Based on pseudocode from https://pseudoeditor.com/guides/binary-search
    """
    left = 0
    right = len(seq_offset) - 1
    while left <= right:
        mid = (left + right) // 2
        min_char_ind, max_char_ind = seq_offset[mid] 
        if min_char_ind <= char and char < max_char_ind:
            return mid
        elif max_char_ind <= char:
            left = mid + 1
        else:
            right = mid - 1
    return -1  # This should not be returned under normal use.


def _set_ner_ids_for_tag(
        ner_ids: list[list[int]],
        start_seq: int, 
        start_index_in_seq: int,
        end_seq: int,
        end_index_in_seq: int,
        label2id: dict[str, int],
        ner_tag: str
        ) -> None:
    """
    After the locations of the tokens corresponding to a HTML tag have been found, 
    mark within `ner_ids` the appropriate NER tags at the locations corresponding
    to the tokens' locations.
    """
    ner_ids[start_seq][start_index_in_seq] = label2id[f"B-{ner_tag}"]
    i_ner_id = label2id[f"I-{ner_tag}"]
    seq, ind = start_seq, start_index_in_seq + 1
    while seq < end_seq or ind <= end_index_in_seq:
        if len(ner_ids[seq]) <= ind:
            seq += 1
            ind = 0
        else:
            ner_ids[seq][ind] = i_ner_id 
            ind += 1
    


def def_or_notat_from_html_tag(
        tag: bs4.element.Tag
        ) -> Union[str, None]:
    """
    Can be passed as the `ner_tag_from_html_tag` argument in `tokenize_html_data`
    for the purposes of compiling a dataset for definition and notation
    identification.

    The strings f"I-{output}" and f"B-{output}" are valid ner_tags. To use for 
    """
    if "definition" in tag.attrs:
        return "definition"
    elif "notation" in tag.attrs:
        return "notation"
    return None  # If the HTML tag carries neither definition nor notation data.

In [ ]:
#| hide
test_eq(_min_max_char_ind_for_seq([(0,0), (1,3), (3,4), (4,7), (7,15), (0,0)]), (1,15))

offsets = [[(0,0), (0,3), (4,5), (5,6), (6,7), (7,8), (8,9),],
           [(10,12), (13,14), (15,18), (18,24)],
           [(25,28), (29,35), (36,42), ]]
test_eq(_search_seq_ind_for_char(offsets, 0), 0)
test_eq(_search_seq_ind_for_char(offsets, 1), 0)
test_eq(_search_seq_ind_for_char(offsets, 5), 0)
test_eq(_search_seq_ind_for_char(offsets, 8), 0)
# I don't think that character index 9 is something that I need to worry about.
test_eq(_search_seq_ind_for_char(offsets, 10), 1)
test_eq(_search_seq_ind_for_char(offsets, 23), 1)
test_eq(_search_seq_ind_for_char(offsets, 25), 2)
test_eq(_search_seq_ind_for_char(offsets, 41), 2)

We continue with an example using the HTML data from the example for the `html_data_from_note` function.

In [ ]:
mf = MarkdownFile.from_string(
    r"""---
aliases: []
tags: []
---
# Galois group of a separable and normal finite field extension

Let $L/K$ be a separable and normal finite field extension. Its <b definition="Galois group of a separable and normal finite field extension">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span> is...

# Galois group of a separable and normal profinite field extension

In fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that
$L = \varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its **Galois group** **$\operatorname{Gal}(L/K)$**

# See Also
# Meta
## References and Citations
""")

html_data = html_data_from_note(mf, vault=None, note_name=None)
print(html_data)

assert html_data['note_name'] is None

{'note_name': None, 'raw_text': 'Let $L/K$ be a separable and normal finite field extension. Its Galois group $\\operatorname{Gal}(L/K)$ is...\n\nIn fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that\n$L = \\varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its Galois group $\\operatorname{Gal}(L/K)$\n', 'tags': [HTMLTagWithIndices(tag=<b definition="Galois group of a separable and normal finite field extension">Galois group</b>, start=64, end=76), HTMLTagWithIndices(tag=<span notation="">$\operatorname{Gal}(L/K)$</span>, start=77, end=102), HTMLTagWithIndices(tag=<b definition="">Galois group</b>, start=330, end=342), HTMLTagWithIndices(tag=<span notation="">$\operatorname{Gal}(L/K)$</span>, start=343, end=368)]}


In [ ]:
html_data['raw_text']
html_data["tags"]

[HTMLTagWithIndices(tag=<b definition="Galois group of a separable and normal finite field extension">Galois group</b>, start=64, end=76),
 HTMLTagWithIndices(tag=<span notation="">$\operatorname{Gal}(L/K)$</span>, start=77, end=102),
 HTMLTagWithIndices(tag=<b definition="">Galois group</b>, start=330, end=342),
 HTMLTagWithIndices(tag=<span notation="">$\operatorname{Gal}(L/K)$</span>, start=343, end=368)]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
label2id = {
    "O": 0,
    "B-definition": 1,
    "I-definition": 2,
    "B-notation": 3,
    "I-notation": 4
}
tokens, ner_tag_ids = tokenize_html_data(html_data, tokenizer, 510, def_or_notat_from_html_tag, label2id)

For this example, `max_length` is set to 510 (tokens). The string ("Raw text") is not very long, so only one sequence should be present.

In [ ]:
test_eq(len(tokens), 1)
test_eq(len(ner_tag_ids), 1)

Now let us see what has been tagged:

In [ ]:
id2label = {value: key for key, value in label2id.items()}
id2label

{0: 'O',
 1: 'B-definition',
 2: 'I-definition',
 3: 'B-notation',
 4: 'I-notation'}

In [ ]:
for token, ner_tag in zip(tokens[0], ner_tag_ids[0]):
    if ner_tag != 0:
        print(f"{token}\t\t{id2label[ner_tag]}")

gal		B-definition
##ois		I-definition
group		I-definition
$		B-notation
\		I-notation
operator		I-notation
##name		I-notation
{		I-notation
gal		I-notation
}		I-notation
(		I-notation
l		I-notation
/		I-notation
k		I-notation
)		I-notation
$		I-notation
gal		B-definition
##ois		I-definition
group		I-definition
$		B-notation
\		I-notation
operator		I-notation
##name		I-notation
{		I-notation
gal		I-notation
}		I-notation
(		I-notation
l		I-notation
/		I-notation
k		I-notation
)		I-notation
$		I-notation


Let us set `max_length` to be shorter to observe an example of a tokenization of a single text across multiple sequences (Of course, in practice, the max token length would be set to be longer, say around 512 or 1024.):

In [ ]:
token_ids, ner_tag_ids = tokenize_html_data(html_data, tokenizer, 20, def_or_notat_from_html_tag, label2id)

In [ ]:
print(len(token_ids))
print(len(ner_tag_ids))

7
7


In [ ]:
ner_tag_ids

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2],
 [2, 2, 2, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 3, 4, 4],
 [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0]]

## Parse Data from standalone text

Alternatively, data can be gathered from a str with definition and notations marked with some general formatting, as long as a parser for those markings is specified. 

In [ ]:
#| export
def _calculate_clean_indices_and_create_tags(
        original_text: str,
        markings: list[tuple[str, int, int, dict]]
        ) -> tuple[str, list[HTMLTagWithIndices]]:
    """Helper: returns (clean_text, list_of_tags)."""
    results = []
    clean_text_parts = []
    current_raw_idx = 0
    clean_text_len = 0
    soup = bs4.BeautifulSoup("", 'html.parser')

    for content, raw_start, raw_end, attrs in markings:
        # Append text BEFORE the mark
        pre_text = original_text[current_raw_idx:raw_start]
        clean_text_parts.append(pre_text)
        
        pre_text_len = len(pre_text)
        clean_text_len += pre_text_len
        
        # Calculate indices
        tag_start = clean_text_len
        tag_end = tag_start + len(content)
        
        # Create Tag
        tag_name = "b" if "definition" in attrs else "span"
        tag = soup.new_tag(tag_name, **attrs)
        tag.string = content
        results.append(HTMLTagWithIndices(tag, tag_start, tag_end))
        
        # Append the CONTENT of the mark (clean text)
        clean_text_parts.append(content)
        
        clean_text_len += len(content)
        current_raw_idx = raw_end

    # Append remaining text
    clean_text_parts.append(original_text[current_raw_idx:])
    
    return "".join(clean_text_parts), results

In [ ]:
#| export
def extract_html_tag_indices_from_marked_text(
        text: str, # The text containing custom markings (e.g. "Let [NOT:G] be a [DEF:group]").
        marker_parser: Callable[[str], list[tuple[str, int, int, dict]]] # A function that parses the text and returns a list of tuples. Each tuple should contain: (1) The *inner content* of the marked section, (2) The *start index* of the marking in `text`, (3) The *end index* of the marking in `text`, and (4) A dictionary of *attributes* for the HTML tag.
        ) -> list[HTMLTagWithIndices]: # A list of `HTMLTagWithIndices` objects. The start/end indices corresponds to the *inner content's* location in a "clean" version of the text (where markings are removed).
    """
    Extracts a list of HTML tags and their indices from marked text.
    """
    markings = marker_parser(text)
    markings.sort(key=lambda x: x[1])
    _, tags = _calculate_clean_indices_and_create_tags(text, markings)
    return tags

In [ ]:
import re

def bracket_parser(text: str) -> list[tuple[str, int, int, dict]]:
    """
    Parses marks like [DEF:content] and [NOT:content].
    Returns list of (content, start, end, attrs).
    """
    # Regex to find [TYPE:content]
    # Note: Using non-greedy match (.*?) to handle multiple brackets correctly
    pattern = re.compile(r'\[(DEF|NOT):(.*?)]')
    
    results = []
    for match in pattern.finditer(text):
        tag_type = match.group(1)
        content = match.group(2)
        start, end = match.span()
        
        attrs = {}
        if tag_type == 'DEF':
            attrs['definition'] = ''
        else:
            attrs['notation'] = ''
            
        results.append((content, start, end, attrs))
    return results

# --- Test Setup ---
# Marked Text: "The [DEF:Galois group] [NOT:$\operatorname{Gal}(L/K)$] is..."
marked_text = r"The [DEF:Galois group] [NOT:$\operatorname{Gal}(L/K)$] is..."

# "Clean" Text (what indices refer to): 
# "The Galois group $\operatorname{Gal}(L/K)$ is..."
# Indices Breakdown:
# "The " -> 0-4
# "Galois group" -> 4-16 (Length 12)
# " " -> 16-17
# "$\operatorname{Gal}(L/K)$" -> 17-42 (Length 25)
# " is..." -> 42...

tag_data = extract_html_tag_indices_from_marked_text(marked_text, bracket_parser)

# --- Assertions ---
assert len(tag_data) == 2, f"Expected 2 tags, got {len(tag_data)}"

# 1. Check Definition Tag ("Galois group")
defn = tag_data[0]
assert defn.tag.string == "Galois group"
assert "definition" in defn.tag.attrs
assert defn.start == 4
assert defn.end == 16, f"Expected end 16, got {defn.end}"

# 2. Check Notation Tag ("$\operatorname{Gal}(L/K)$")
notat = tag_data[1]
expected_math = r"$\operatorname{Gal}(L/K)$"
assert notat.tag.string == expected_math
assert "notation" in notat.tag.attrs
assert notat.start == 17, f"Expected start 17, got {notat.start}"
assert notat.end == 42, f"Expected end 42, got {notat.end}"

print("html_tag_data_from_marked_text (Galois example) passed!")


html_tag_data_from_marked_text (Galois example) passed!


In [ ]:
#| export
def html_data_from_marked_text(
        text: str, # The text containing custom markings (e.g. "Let [NOT:G] be a [DEF:group]").
        marker_parser: Callable[[str], list[tuple[str, int, int, dict]]] # A function that parses the text and returns a list of tuples. Each tuple should contain: (1) The *inner content* of the marked section, (2) The *start index* of the marking in `text`, (3) The *end index* of the marking in `text`, and (4) A dictionary of *attributes* for the HTML tag.
        ) -> StrAndHTMLTagsWithIndices: # An object containing the "clean" text (markings removed) and the list of HTML tags with their indices in that clean text.
    """
    Creates an `StrAndHTMLTagsWithIndices` object from text with custom markings.
    """
    markings = marker_parser(text)
    markings.sort(key=lambda x: x[1])
    clean_text, tags = _calculate_clean_indices_and_create_tags(text, markings)
    
    return StrAndHTMLTagsWithIndices(clean_text, tags)

In [ ]:
#| hide
import re

def bracket_parser(text: str) -> list[tuple[str, int, int, dict]]:
    """Parses [DEF:content] and [NOT:content]."""
    pattern = re.compile(r'\[(DEF|NOT):(.*?)]')
    results = []
    for match in pattern.finditer(text):
        tag_type = match.group(1)
        content = match.group(2)
        start, end = match.span()
        attrs = {'definition' if tag_type == 'DEF' else 'notation': ''}
        results.append((content, start, end, attrs))
    return results

# --- Test Setup ---
# Marked Text: "The [DEF:Galois group] [NOT:$\operatorname{Gal}(L/K)$] is..."
marked_text = r"The [DEF:Galois group] [NOT:$\operatorname{Gal}(L/K)$] is..."

# Expected "Clean" Text (markings stripped, content kept)
expected_clean_text = r"The Galois group $\operatorname{Gal}(L/K)$ is..."

# --- Run Function ---
data_obj = html_data_from_marked_text(marked_text, bracket_parser)

print(data_obj)
# --- Assertions ---

# 1. Check Clean Text
assert data_obj.raw_text == expected_clean_text, \
    f"Clean text mismatch.\nExpected: '{expected_clean_text}'\nGot:      '{data_obj.raw_text}'"

# 2. Check Tag Count
assert len(data_obj.tags) == 2, f"Expected 2 tags, got {len(data_obj.tags)}"

# 3. Check Definition Tag ("Galois group")
# Indices in clean text: "The " (4) -> start 4
# "Galois group" (length 12) -> end 16
defn = data_obj.tags[0]
assert defn.tag.string == "Galois group"
assert "definition" in defn.tag.attrs
assert defn.start == 4
assert defn.end == 16

# 4. Check Notation Tag ("$\operatorname{Gal}(L/K)$")
# Indices in clean text:
# "The Galois group " (17 chars: 4 + 12 + 1 space) -> start 17?
# Wait: "The " (4) + "Galois group" (12) + " " (1) = 17.
# So Notation starts at 17.
# Content: "$\operatorname{Gal}(L/K)$" (Length 25)
# End: 17 + 25 = 42.
notat = data_obj.tags[1]
expected_math = r"$\operatorname{Gal}(L/K)$"
assert notat.tag.string == expected_math
assert "notation" in notat.tag.attrs
assert notat.start == 17
assert notat.end == 42

print("html_data_from_marked_text (Galois example) passed!")


StrAndHTMLTagsWithIndices(raw_text='The Galois group $\\operatorname{Gal}(L/K)$ is...', tags=[HTMLTagWithIndices(tag=<b definition="">Galois group</b>, start=4, end=16), HTMLTagWithIndices(tag=<span notation="">$\operatorname{Gal}(L/K)$</span>, start=17, end=42)])
html_data_from_marked_text (Galois example) passed!


### Miscellaneous parse formatting

The following is intended to parse latex code of the author's writing for definition and notation markings

In [ ]:
#| export
def latex_highlight_parser(text: str) -> list[tuple[str, int, int, dict]]:
    """
    Parses LaTeX highlighting commands (\\hldef, \\hl, \\hlin, \\hlalign) to identify
    definitions and notations.
    
    This function is designed to be passed as the `marker_parser` argument to
    `html_data_from_marked_text`.

    It handles nested braces using recursive regex and strips surrounding whitespace 
    from the inner content (e.g., stripping padding newlines in `\\hlalign{\\n ... \\n}`).
    It also implements special logic for `\\hlin` to expand the notation range to 
    include surrounding `$$` delimiters if present.

    Returns
    -------
    list[tuple[str, int, int, dict]]
        A list of tuples representing the found markings. Each tuple contains:
        1. **Content** (`str`): The inner text of the marking (stripped of padding whitespace).
           This is the text that will remain in the "clean" output and be wrapped by the tag.
        2. **Start Index** (`int`): The start index of the *entire marking wrapper* 
           (e.g., the index of `\\`) in the original `text`.
        3. **End Index** (`int`): The end index of the *entire marking wrapper* 
           (e.g., the index after `}`) in the original `text`.
        4. **Attributes** (`dict`): A dictionary of HTML attributes, e.g. 
           `{'definition': ''}` or `{'notation': ''}`.
    """
    pattern = r'\\(hldef|hlalign|hlin|hl)\s*(\{(?:[^{}]++|(?2))*\})'
    results = []
    
    for match in regex.finditer(pattern, text):
        cmd_type = match.group(1)
        full_brace_group = match.group(2)
        raw_inner = full_brace_group[1:-1]
        match_start, match_end = match.span()
        
        # Strip padding whitespace from content
        lstripped = raw_inner.lstrip()
        stripped = lstripped.rstrip()
        content = stripped
        
        start = match_start
        end = match_end
        attrs = {'definition' if cmd_type == 'hldef' else 'notation': ''}
        
        if cmd_type == 'hlin':
            # 1. Scan Backwards for start $$
            p = start - 1
            while p >= 0 and text[p].isspace(): p -= 1
            
            # Check if we hit $$ immediately (ignoring space)
            if p >= 1 and text[p] == '$' and text[p-1] == '$':
                found_start_dollars = True
                new_start = p - 1
                # BUG WAS HERE: prefix = "" 
                # FIX: Capture the text between $$ (p+1) and \hlin (start)
                prefix = text[p+1 : start] 
            else:
                found_start_dollars = False
                prefix = "" # Initialize for safety

            
            # 2. Scan Forwards for end $$
            # We want to allow punctuation like "." or "," between } and $$
            # E.g. $$ \hlin{x}. $$
            
            q = end
            suffix = ""
            found_end_dollars = False
            
            # Heuristic: Scan forward for a limited distance or until $$
            # We capture everything between } and $$ into 'suffix'
            # Stop if we hit a newline (safeguard)
            temp_q = q
            while temp_q < len(text) - 1:
                if text[temp_q] == '\n': break
                
                if text[temp_q] == '$' and text[temp_q+1] == '$':
                    found_end_dollars = True
                    new_end = temp_q + 2
                    # The text between original end and $$ is the suffix
                    suffix = text[end:temp_q]
                    # Clean the suffix? Usually we just want to include it.
                    # e.g. suffix might be ". " (period and space)
                    # We usually trim the space before the $$, but keeping it is safer for fidelity.
                    break
                temp_q += 1
            
            # Only apply expansion if we found BOTH start and end $$
            # AND (optionally) if we found the start $$ directly. 
            # (Does it make sense to have content BEFORE \hlin? e.g. $$ x = \hlin{y} $$?
            #  If so, we should scan backwards for $$ similarly to how we scanned forwards).
            
            # Let's implement symmetric scanning for robustness.
            
            # RE-TRY Backward Scan with content capture
            if not found_start_dollars:
                 temp_p = start - 1
                 while temp_p >= 1:
                     if text[temp_p] == '\n': break
                     if text[temp_p] == '$' and text[temp_p-1] == '$':
                         found_start_dollars = True
                         new_start = temp_p - 1
                         prefix = text[temp_p+1 : start] # Content between $$ and \hlin
                         break
                     temp_p -= 1

            if found_start_dollars and found_end_dollars:
                # We found $$ ... \hlin{...} ... $$
                # We want the tag to cover the WHOLE thing: "$$ prefix content suffix $$"
                # And we want to remove the WHOLE thing from the text.
                
                # Careful: The 'prefix' and 'suffix' currently contain the raw characters 
                # from the marked text (including potentially ignored spaces).
                # We should probably strip excessive padding next to the $$ inside the tag?
                # Standard convention: $$ content $$ -> Tag content "$$ content $$"
                
                # Let's just assemble it raw to preserve user's punctuation/spacing logic,
                # then maybe strip outer edges if needed.
                
                # Current 'content' is stripped inner content of \hlin.
                # 'prefix' is text between $$ and \hlin.
                # 'suffix' is text between \hlin and $$.
                
                # Example: $$ \hlin{K}. $$
                # prefix = " "
                # content = "K"
                # suffix = ". "
                # Result: "$$ K. $$"
                
                full_content = f"$${prefix}{content}{suffix}$$"
                
                content = full_content
                start = new_start
                end = new_end

        results.append((content, start, end, attrs))
        
    return results
    # pattern = r'\\(hldef|hlalign|hlin|hl)\s*(\{(?:[^{}]++|(?2))*\})'
    
    # results = []
    
    # for match in regex.finditer(pattern, text):
    #     cmd_type = match.group(1)
    #     full_brace_group = match.group(2)
        
    #     # Raw content inside braces (e.g. " \n Content \n ")
    #     raw_inner = full_brace_group[1:-1]
        
    #     # Calculate indices relative to the 'text' string
    #     # Match span covers `\hl{...}`
    #     match_start, match_end = match.span()
        
    #     # We want to identify where the "real" content starts/ends inside the match
    #     # Start of raw_inner is: match_end - 1 (closing brace) - len(raw_inner)
    #     # Actually easier: find start of {
    #     brace_start_idx = match.start(2) # Index of {
    #     inner_start_idx = brace_start_idx + 1
        
    #     # Find leading/trailing whitespace length
    #     lstripped = raw_inner.lstrip()
    #     leading_ws_len = len(raw_inner) - len(lstripped)
        
    #     stripped = lstripped.rstrip()
    #     trailing_ws_len = len(lstripped) - len(stripped)
        
    #     content = stripped
    #     attrs = {'definition' if cmd_type == 'hldef' else 'notation': ''}
        
    #     # 1. Base Range: The command wrapper `\cmd{` and `}`.
    #     # We want to effectively say:
    #     # "Remove `\cmd{ \n`", keep `Content`, "Remove `\n }`".
    #     # But our interface only supports "Remove `Range`, Insert `Content`".
        
    #     # If we return `start=match_start`, `end=match_end`, `content=stripped`:
    #     # "Remove `\hl{ \n Content \n }`. Insert `Content`."
    #     # Result Clean Text: `Content`. (Newlines LOST).
        
    #     # If the user WANTS those newlines preserved in the clean text (outside the tag),
    #     # we have a problem. The current architecture assumes "Marked Region" -> "Tag".
    #     # It doesn't support "Marked Region" -> "Prefix + Tag + Suffix".
        
    #     # DECISION: 
    #     # For `\hlalign`, users usually write:
    #     # \hlalign{
    #     # \begin{align}
    #     # ...
    #     # \end{align}
    #     # }
    #     # They EXPECT the clean text to contain the newlines so the align renders correctly?
    #     # Actually, LaTeX doesn't care about the surrounding newlines much.
    #     # `\begin{align}...\end{align}` is valid without extra newlines.
    #     # So losing the newlines inside the braces is probably ACCEPTABLE and cleaner.
        
    #     # HOWEVER, if `$$ \hlin{ x } $$` -> `$$x$$`. Losing spaces is fine.
        
    #     # Special Logic for \hlin ($$) remains...
        
    #     # Let's apply the stripping logic:
        
    #     start = match_start
    #     end = match_end
        
    #     if cmd_type == 'hlin':
    #         # ... (Logic to expand to $$ remains same, but apply to `stripped` content) ...
    #         # Re-implementing simplified logic for clarity in this snippet:
    #         p = start - 1
    #         while p >= 0 and text[p].isspace(): p -= 1
    #         if p >= 1 and text[p] == '$' and text[p-1] == '$':
    #             new_start = p - 1
    #             q = end
    #             while q < len(text) and text[q].isspace(): q += 1
    #             if q < len(text) - 1 and text[q] == '$' and text[q+1] == '$':
    #                 new_end = q + 2
    #                 content = f"$${content}$$"
    #                 start = new_start
    #                 end = new_end

    #     results.append((content, start, end, attrs))
        
    # return results


In [ ]:
from bs4 import BeautifulSoup

# --- Setup ---
# Input Text with mixed highlighting
# 1. Definition: \hldef{Galois group}
# 2. Inline Notation: \hl{$\operatorname{Gal}(L/K)$}
# 3. Display Notation: $$\hlin{x^2}$$
marked_text = r"The \hldef{Galois group} \hl{$\operatorname{Gal}(L/K)$} is defined. Display: $$\hlin{x^2}$$"

# Expected Clean Text
expected_clean = r"The Galois group $\operatorname{Gal}(L/K)$ is defined. Display: $$x^2$$"

# --- Run ---
data_obj = html_data_from_marked_text(marked_text, latex_highlight_parser)

# --- Assertions ---

# 1. Check Clean Text
assert data_obj.raw_text == expected_clean, \
    f"Clean text mismatch.\nExpected: {expected_clean}\nGot:      {data_obj.raw_text}"

# 2. Check Tag Count (3 tags)
assert len(data_obj.tags) == 3

# 3. Check Definition
defn = data_obj.tags[0]
assert defn.tag.string == "Galois group"
assert "definition" in defn.tag.attrs
# "The " (4) -> start 4
assert defn.start == 4

# 4. Check Inline Notation
notat_inline = data_obj.tags[1]
assert notat_inline.tag.string == r"$\operatorname{Gal}(L/K)$"
assert "notation" in notat_inline.tag.attrs
# "The Galois group " (17) -> start 17
assert notat_inline.start == 17

# 5. Check Display Notation
# Clean text segment: " is defined. Display: $$"
# "The Galois group $\operatorname{Gal}(L/K)$" (length 17 + 25 = 42)
# " is defined. Display: $$" (length 22)
# Start index should be 42 + 22 = 64
notat_display = data_obj.tags[2]
assert notat_display.tag.string == "$$x^2$$"
assert "notation" in notat_display.tag.attrs

print("latex_highlight_parser test passed!")


latex_highlight_parser test passed!


In [ ]:
print(data_obj.raw_text)
print(data_obj.tags)

The Galois group $\operatorname{Gal}(L/K)$ is defined. Display: $$x^2$$
[HTMLTagWithIndices(tag=<b definition="">Galois group</b>, start=4, end=16), HTMLTagWithIndices(tag=<span notation="">$\operatorname{Gal}(L/K)$</span>, start=17, end=42), HTMLTagWithIndices(tag=<span notation="">$$x^2$$</span>, start=64, end=71)]


In [ ]:

# --- Setup ---
# 1. Nested Braces: \hl{$\{x\}$}
# 2. Display Math: $$\hlin{x^2}$$
# 3. Align: \hlalign{\begin{align}a&=b\end{align}}

marked_text = (
    r"Set \hl{$\{x\}$}. "
    r"Display: $$\hlin{x^2}$$ "
    r"Align: \hlalign{\begin{align}a&=b\end{align}}"
)

# Expected Clean Text
# 1. $\begin{Bmatrix}x\end{Bmatrix}$ (restored)
# 2. $$x^2$$ (restored, tag wraps entire thing)
# 3. \begin{align}a&=b\end{align} (restored)
expected_clean = (
    r"Set $\{x\}$. "
    r"Display: $$x^2$$ "
    r"Align: \begin{align}a&=b\end{align}"
)

# --- Run ---
data_obj = html_data_from_marked_text(marked_text, latex_highlight_parser)

# --- Assertions ---
# 1. Clean Text
assert data_obj.raw_text == expected_clean, \
    f"Clean text mismatch.\nExpected: '{expected_clean}'\nGot:      '{data_obj.raw_text}'"

# 2. Nested Brace Tag
tag_nested = data_obj.tags[0]
assert tag_nested.tag.string == r"$\{x\}$"
assert "notation" in tag_nested.tag.attrs

# 3. Display Math Tag
# Should wrap "$$x^2$$" because parser expanded to include $$
tag_display = data_obj.tags[1]
assert tag_display.tag.string == r"$$x^2$$"
assert "notation" in tag_display.tag.attrs

# 4. Align Tag
tag_align = data_obj.tags[2]
assert tag_align.tag.string == r"\begin{align}a&=b\end{align}"
assert "notation" in tag_align.tag.attrs

print("Robust parser tests passed!")
print(data_obj)

Robust parser tests passed!
StrAndHTMLTagsWithIndices(raw_text='Set $\\{x\\}$. Display: $$x^2$$ Align: \\begin{align}a&=b\\end{align}', tags=[HTMLTagWithIndices(tag=<span notation="">$\{x\}$</span>, start=4, end=11), HTMLTagWithIndices(tag=<span notation="">$$x^2$$</span>, start=22, end=29), HTMLTagWithIndices(tag=<span notation="">\begin{align}a&amp;=b\end{align}</span>, start=37, end=65)])


In [ ]:
#| hide
#| notest

# --- Setup ---
# Input has padding newlines inside the braces:
# {
#   \begin{align} ... \end{align}
# }
marked_text = r"""
Here is some text.
\hlalign{
\begin{align}
asdf
\end{align}
}
End of text.
"""

# Expected Clean Text:
# The `\hlalign{` and `}` are removed.
# Crucially, the padding newlines inside the braces are ALSO removed by the strip() logic.
# So we expect NO newline between "Here is some text." and "\begin{align}",
# unless there was one outside the command (which there is: line 2 has a newline).
#
# Original:
# Line 1: "Here is some text."
# Line 2: "\hlalign{" -> Replaced by "\begin{align}" (start of content)
# ... content ...
# Line 6: "}" -> Replaced by "" (end of content)
# Line 7: "End of text."

# So the clean text will effectively collapse the structure slightly.
# "Here is some text.\n\begin{align}\nasdf\n\end{align}\nEnd of text."
expected_clean = (
    "Here is some text.\n"
    r"\begin{align}" + "\n"
    r"asdf" + "\n"
    r"\end{align}" + "\n"
    "End of text."
)

# --- Run ---
data_obj = html_data_from_marked_text(marked_text, latex_highlight_parser)

# --- Assertions ---

# 1. Check Clean Text
# If the parser failed to strip padding, we would see extra newlines (e.g. \n\n\begin...)
assert data_obj.raw_text.strip() == expected_clean.strip(), \
    f"Clean text mismatch.\nExpected:\n{repr(expected_clean)}\nGot:\n{repr(data_obj.raw_text)}"

# 2. Check Tag Content
tag_align = data_obj.tags[0]
tag_content = tag_align.tag.string

# Verify NO leading/trailing newlines in the tag content itself
# (It should start immediately with \begin and end with \end)
assert tag_content.startswith(r"\begin{align}"), f"Tag content has leading garbage: {repr(tag_content[:20])}"
assert tag_content.endswith(r"\end{align}"), f"Tag content has trailing garbage: {repr(tag_content[-20:])}"

# Verify INTERNAL newlines are preserved (between begin and asdf)
assert "\nasdf\n" in tag_content, "Internal newlines were incorrectly stripped!"

print("Multiline hlalign padding test passed!")


Multiline hlalign padding test passed!


Here are some examples of `latex_highlight_parser` in the context of `html_data_from_marked_text`

In [ ]:
# from trouver.machinelearning.tokenize.def_and_notat_token_classification import (
#     html_data_from_marked_text, 
#     latex_highlight_parser
# )

# Input text with marking commands
marked_text = r"The \hldef{Galois group} \hl{$\operatorname{Gal}(L/K)$} is defined."

# Process the text
data = html_data_from_marked_text(marked_text, latex_highlight_parser)

# 1. Clean Text Verification
# The commands \hldef{...} and \hl{...} are stripped, leaving only the content.
expected_clean = r"The Galois group $\operatorname{Gal}(L/K)$ is defined."
assert data.raw_text == expected_clean

# 2. Tag Verification
# Tag 1: Definition
def_tag = data.tags[0]
assert def_tag.tag.string == "Galois group"
assert "definition" in def_tag.tag.attrs
# "The " is 4 chars long, so start is 4.
assert def_tag.start == 4 
assert def_tag.end == 4 + len("Galois group")

# Tag 2: Notation
not_tag = data.tags[1]
assert not_tag.tag.string == r"$\operatorname{Gal}(L/K)$"
assert "notation" in not_tag.tag.attrs
# Starts after "The Galois group " (17 chars)
assert not_tag.start == 17

print("Example 1 Passed")


Example 1 Passed


In [ ]:
# Input: Display math where \hlin wraps the inner equation
# Note: The parser handles the spaces around the `\hlin` command inside the $$
marked_text = r"The kernel is trivial: $$ \hlin{ K = \{e\} } $$"

data = html_data_from_marked_text(marked_text, latex_highlight_parser)

# 1. Clean Text Verification
# The result should look like standard LaTeX display math.
# The spaces inside the braces are stripped by the parser, but the spaces
# between $$ and the content depend on how we reconstruct it.
# Our parser returns content "$$K = \{e\}$$" (with $$ added).
# And it consumes the original "$$ \hlin{...} $$".
# So the clean text is just the content inserted at that spot.
expected_clean = r"The kernel is trivial: $$ K = \{e\} $$"
assert data.raw_text == expected_clean

# 2. Tag Verification
tag = data.tags[0]
# The tag should wrap the WHOLE math block, including $$
assert tag.tag.string == r"$$ K = \{e\} $$"
assert "notation" in tag.tag.attrs

print("Example 2 Passed")


Example 2 Passed


In [ ]:
#| hide

# --- Setup ---
# Case: Punctuation inside $$ but outside \hlin
marked_text = r"The kernel is trivial: $$ \hlin{ K = \{e\} }. $$"

data = html_data_from_marked_text(marked_text, latex_highlight_parser)

# 1. Clean Text Verification
# Original text logic would normally produce:
# "The kernel is trivial: $$ K = \{e\} . $$" (spaces preserved from prefix/suffix)
#
# Let's trace the parser:
# prefix = " " (between $$ and \hlin)
# content = "K = \{e\}"
# suffix = ". " (between } and $$)
# result = "$$ K = \{e\}. $$"
expected_clean = r"The kernel is trivial: $$ K = \{e\}. $$"

assert data.raw_text == expected_clean, \
    f"Clean Text Mismatch.\nExp: '{expected_clean}'\nGot: '{data.raw_text}'"

# 2. Tag Verification
tag = data.tags[0]
tag_str = tag.tag.string

# Ensure the tag wraps the whole $$ block
assert tag_str.startswith("$$")
assert tag_str.endswith("$$")
assert r"K = \{e\}" in tag_str
# Ensure the period is INSIDE the tag
assert "." in tag_str
# Ensure the tag covers the full range
assert tag.start == 23 # Index of $$ start in "The kernel is trivial: "

print("Typo/Punctuation inside $$ test passed!")


Typo/Punctuation inside $$ test passed!


In [ ]:
#| hide
# Input: Multi-line align block with padding newlines inside the command
marked_text = r"""Consider the sequence:
\hlalign{
\begin{align}
0 \to A \to B \to 0
\end{align}
}
It is exact."""

data = html_data_from_marked_text(marked_text, latex_highlight_parser)

# 1. Clean Text Verification
# Padding newlines inside \hlalign{...} are stripped.
# The newline AFTER "sequence:" remains.
# The newline BEFORE "It is exact" remains.
expected_clean = r"""Consider the sequence:
\begin{align}
0 \to A \to B \to 0
\end{align}
It is exact."""

# Strip checks to ignore potential leading/trailing whitespace difference in the file itself
assert data.raw_text.strip() == expected_clean.strip()

# 2. Tag Verification
tag = data.tags[0]
content = tag.tag.string

# Ensure the tag content preserves the internal structure
assert r"\begin{align}" in content
assert r"0 \to A" in content
assert r"\end{align}" in content
assert content.startswith(r"\begin{align}")
assert content.endswith(r"\end{align}")
# Ensure internal newlines are preserved
assert "\n" in content
# Ensure tag is marked as notation
assert "notation" in tag.tag.attrs

print("Example 3 Passed")


Example 3 Passed


## Augmenting data

In [ ]:
#| export
def _split_text_by_html_data_parts(
        # text_tags_and_locations = StrAndHTMLTagsWithIndices
        datapoint: HTMLData
        ) -> list[tuple[str, Union[bs4.element.Tag, None]]]:
    r"""
    Helper function to `augment_html_data`.
    """
    to_return: list[tuple[str, Union[bs4.element.Tag, None]]] = []
    split_points: list[int] = []
    for tag_ind in datapoint['tags']:
        split_points.append(tag_ind.start)
        split_points.append(tag_ind.end)
    pieces: list[str] = split_string_at_indices(datapoint['raw_text'], split_points)
    for i, piece in enumerate(pieces):
        if i % 2 == 0:
            to_return.append((piece, None))
        else:
            to_return.append((piece, datapoint['tags'][int(i/2)].tag))
    return to_return


In [ ]:
#|hide
sample_text = r"""# Galois group of a separable and normal finite field extension

Let $L/K$ be a separable and normal finite field extension. Its <b definition="Galois group of a separable and normal finite field extension">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span> is...

# See Also
# Meta
## References and Citations
"""

sample_str_and_html_tags_with_indices: StrAndHTMLTagsWithIndices = remove_html_tags_in_text(sample_text)
sample_html_data = HTMLData(note_name=None, raw_text=sample_str_and_html_tags_with_indices.raw_text, tags=sample_str_and_html_tags_with_indices.tags)
output = _split_text_by_html_data_parts(sample_html_data)
assert isinstance(output[1][1], bs4.element.Tag)
output


[('# Galois group of a separable and normal finite field extension\n\nLet $L/K$ be a separable and normal finite field extension. Its ',
  None),
 ('Galois group',
  <b definition="Galois group of a separable and normal finite field extension">Galois group</b>),
 (' ', None),
 ('$\\operatorname{Gal}(L/K)$',
  <span notation="">$\operatorname{Gal}(L/K)$</span>),
 (' is...\n\n# See Also\n# Meta\n## References and Citations\n', None)]

In [ ]:
#| hide
sample_text = r"""<b definition="">Hello</b> asdf <b notation=""> $\operatorname{Gal}$</b>"""
sample_str_and_html_tags_with_indices: StrAndHTMLTagsWithIndices = remove_html_tags_in_text(sample_text)
sample_html_data = HTMLData(note_name=None, raw_text=sample_str_and_html_tags_with_indices.raw_text, tags=sample_str_and_html_tags_with_indices.tags)
output = _split_text_by_html_data_parts(sample_html_data)
assert isinstance(output[1][1], bs4.element.Tag)
assert isinstance(output[3][1], bs4.element.Tag)
test_is(output[4][1], None)
output

[('', None),
 ('Hello', <b definition="">Hello</b>),
 (' asdf ', None),
 (' $\\operatorname{Gal}$', <b notation=""> $\operatorname{Gal}$</b>),
 ('', None)]

In [ ]:
print(sample_str_and_html_tags_with_indices.tags)

[HTMLTagWithIndices(tag=<b definition="">Hello</b>, start=0, end=5), HTMLTagWithIndices(tag=<b notation=""> $\operatorname{Gal}$</b>, start=11, end=32)]


In [ ]:
#| export
def augment_html_data(
        datapoint: HTMLData,
        num_augmentation_sets: int = 1, # Each augmentation set consists of an augmentation with low, medium, and high probability modifications.
        seed: Optional[int] = None
        ) -> list[HTMLData]:
    r"""Augment a given datapoint for HTML tagging.

    """
    augmented_datapoints: list[HTMLData] = []
    pieces: list[tuple[str, Union[bs4.element.Tag, None]]] = _split_text_by_html_data_parts(
        datapoint)
    if seed is not None:
        random.seed(seed)
    for _ in range(num_augmentation_sets):
        augmented_datapoints.append(
            _augment_html_data_once(pieces, 'low', datapoint['note_name']))
        augmented_datapoints.append(
            _augment_html_data_once(pieces, 'mid', datapoint['note_name']))
        augmented_datapoints.append(
            _augment_html_data_once(pieces, 'hi', datapoint['note_name']))
        # augmented_datapoints.append(_augment_html_data_once(pieces, 'high'))
    return augmented_datapoints


def _augment_html_data_once(
        pieces: list[tuple[str, Union[bs4.element.Tag, None]]],
        modification: Literal['low', 'mid', 'high'],
        note_name: str,
        ) -> HTMLData:
    methods = [
        # (push_dollar_signs,0.2),
        (remove_font_styles_at_random, 0.1), (change_font_styles_at_random, 0.2), (change_greek_letters_at_random, 0.1), 
        (remove_math_keywords,0.1), (random_latex_command_removal,0.2),
        (random_word_removal,0.1), (dollar_sign_manipulation,0.05),
        (random_char_modification,0.001)]
    if modification == 'low':
        method_inclusion_chance = 0.3
        scale = 0.5
    elif modification == 'mid':
        method_inclusion_chance = 0.5
        scale = 1.0
    else:
        method_inclusion_chance = 0.8
        scale = 1.5
    
    random_methods = []
    def create_method(method, p, scale):
        return lambda x: method(x, p=p*scale)
    for method, p in methods:
        if random.random() < method_inclusion_chance:
            random_methods.append(create_method(method, p, scale))
    # random_methods = [
    #     lambda x: method(x, p=p*scale) for method, p in methods if random.random() < method_inclusion_chance]
    augmented_pieces = [
        (augment_text(text, random_methods), copy.copy(tag))
        for text, tag in pieces]
    accumulated_len: int = 0
    accumulated_text: str = ""
    tags_with_indices: list[HTMLTagWithIndices] = []
    for text, tag in augmented_pieces:
        accumulated_text = f'{accumulated_text}{text}'
        if tag:
            tag.string = text
            tags_with_indices.append(HTMLTagWithIndices(
                tag, accumulated_len, accumulated_len + len(text)))
        accumulated_len += len(text)
    return HTMLData(note_name=note_name, raw_text=accumulated_text, tags=tags_with_indices)

The `augment_html_data` can be used to augment definition and notation token classification data gathered via `html_data_from_note`.

In [ ]:
mf = MarkdownFile.from_string(
    r"""---
aliases: []
tags: []
---
# Galois group of a separable and normal finite field extension

Let $L/K$ be a separable and normal finite field extension. Its <b definition="Galois group of a separable and normal finite field extension">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span> is...

# Galois group of a separable and normal profinite field extension

In fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that
$L = \varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its **Galois group** **$\operatorname{Gal}(L/K)$**

# See Also
# Meta
## References and Citations
""")

html_data: HTMLData = html_data_from_note(mf)
output = augment_html_data(html_data,seed=None)
html_data, output
# html_data

({'note_name': None,
  'raw_text': 'Let $L/K$ be a separable and normal finite field extension. Its Galois group $\\operatorname{Gal}(L/K)$ is...\n\nIn fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that\n$L = \\varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its Galois group $\\operatorname{Gal}(L/K)$\n',
  'tags': [HTMLTagWithIndices(tag=<b definition="Galois group of a separable and normal finite field extension">Galois group</b>, start=64, end=76),
   HTMLTagWithIndices(tag=<span notation="">$\operatorname{Gal}(L/K)$</span>, start=77, end=102),
   HTMLTagWithIndices(tag=<b definition="">Galois group</b>, start=330, end=342),
   HTMLTagWithIndices(tag=<span notation="">$\operatorname{Gal}(L/K)$</span>, start=343, end=368)]},
 [{'note_name': None,
   'raw_text': 'Let $L/K$ be a separable and normal finite field extension. Its Galois group $\\operatorname{Gal}(L/K)$ is...\n\nIn f

## Gathering data 

The following is sample code to then gather data for definition/notation identification

In [ ]:
#| notest

# TODO: test

notes = [] # Replace with actual notes
vault = '' # Replace with actual vault

# vault = 'C:' # Replace with actual vault
# notes = [] # Replace with actual notes

html_data = [html_data_from_note(note, vault) for note in notes]
max_length = 1022

tokenized_html_data = [tokenize_html_data(html_locus, tokenizer, max_length, def_or_notat_from_html_tag, label2id) for html_locus in html_data]
token_id_data = [token_ids for token_ids, _ in tokenized_html_data]
ner_tag_data = [ner_tag_ids for _, ner_tag_ids in tokenized_html_data]
token_seqs = [token_seq for token_seq in token_ids for token_ids in token_id_data]
ner_tag_seqs = [ner_tag_seq for ner_tag_seq in ner_tag_ids for ner_tag_ids in ner_tag_data]

In [ ]:
#| notest
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
max_length = 1022
label2id = {
    "O": 0,
    "B-definition": 1,
    "I-definition": 2,
    "B-notation": 3,
    "I-notation": 4
} 
id2label = {value: key for key, value in label2id.items()}

In [ ]:
#| notest
note_names, token_seqs, ner_tag_seqs = [], [], []
for html_locus, (token_ids, ner_tag_ids) in zip(html_data, tokenized_html_data):
    note_names.extend([html_locus["Note name"]] * len(token_ids))
    token_seqs.extend(token_ids)
    ner_tag_seqs.extend(ner_tag_ids)

In [ ]:
#| notest
# ner_tags = ClassLabel(names=list(label2id))

# ds = Dataset.from_dict(
#         {"note_name": note_names,
#         "tokens": token_ids,
#         "ner_tags": ner_tag_ids},
#         features=Features(
#             {
#              "note_name": Value(dtype='string'),
#              "tokens": Sequence(Value(dtype='string')),
#              "ner_tags": Sequence(ner_tags)}
#         ))

# ds.save_to_disk(".")

# ds.load_from_disk(".")
    

## Use the trained model

See https://huggingface.co/docs/transformers/tasks/token_classification for training a token classification model.

In [ ]:
# Helper functions

In [ ]:
soup = bs4.BeautifulSoup('', 'html.parser')
tag = soup.new_tag('b', style="border-width:1px;border-style:solid;padding:3px", definition="")
tag.string = 'hi'
tag


<b definition="" style="border-width:1px;border-style:solid;padding:3px">hi</b>

In [ ]:
#| export
def _make_tag(
        text: str,
        entity_type: str # 'definition' or 'notation'
        ) -> bs4.element.Tag:
    """
    Helper function to `_html_tag_data_from_part` and `_consolidate_token_preds`.
    """
    soup = bs4.BeautifulSoup('', 'html.parser')
    if entity_type == 'definition':
        tag = soup.new_tag(
            'b',
            style="border-width:1px;border-style:solid;padding:3px",
            definition="")
    else:
        tag = soup.new_tag(
            'span',
            style="border-width:1px;border-style:solid;padding:3px",
            notation="")
    tag.string = text
    return tag


def _html_tag_data_from_part(
        main_text: str,
        # part: list[dict[str]]) -> tuple[bs4.element.Tag, int, int]:
        part: list[dict[str]] # An item of an output of `_divide_token_preds_into_parts`. Each dict likely contains keys such as `'entity'`, `'score'`, `'index'`, `'word'`, `'start'`, and `'end'`, depending on the model used.
        ) -> tuple[HTMLTagWithIndices]:
    """
    Helper function to `_html_tags_from_token_preds`
    """
    start_token: dict[str] = part[0]
    end_token: dict[str] = part[-1]
    start_char_pos: int = start_token['start']
    end_char_pos: int = end_token['end']

    # Depending on the tokenizer starting spaces might be included in a given token.
    # We exclude such a starting space.
    while main_text[start_char_pos].isspace():
        start_char_pos += 1
    while main_text[end_char_pos-1].isspace():
        end_char_pos -= 1

    # the `'entity'` is either 'I-definition', 'B-definition', 'I-notation',
    # or 'B-notation'
    entity_type = start_token['entity'][2:]
    html_text = main_text[start_char_pos:end_char_pos]
    
    # return (_make_tag(html_text, entity_type), start_char, end_char)
    return HTMLTagWithIndices(_make_tag(html_text, entity_type), start_char_pos, end_char_pos)


In [ ]:
#| hide

main_text = r"Let $I \subset A$ be an ideal. Define its radical by $\sqrt{I}$"

sample_output_1 = _html_tag_data_from_part(
    main_text, [{
        'entity': 'B-definition',
        'score': 0.37319255,
        'index': 25,  # This is moot for the purposes of this test.
        'word': 'radical',
        'start': 42,
        'end': 49
    }])
test_eq(str(sample_output_1.tag), '<b definition="" style="border-width:1px;border-style:solid;padding:3px">radical</b>')

sample_output_2 = _html_tag_data_from_part(
    main_text, [{
        'entity': 'B-notation',
        'score': 0.67021805,
        'index': 27,  # This is moot for the purposes of this test.
        'word': '$',
        'start': 53,
        'end': 54},
    {
        'entity': 'I-notation',
        'score': 0.9748327,
        'index': 28,  # This is moot for the purposes of this test.
        'word': '\\',
        'start': 54,
        'end': 55},
    {
        'entity': 'I-notation',
        'score': 0.9754836,
        'index': 29,  # This is moot for the purposes of this test.
        'word': 'sq',
        'start': 55,
        'end': 57},
    {
        'entity': 'I-notation',
        'score': 0.9750675,
        'index': 30,  # This is moot for the purposes of this test.
        'word': '##rt',
        'start': 57,
        'end': 59},
    {
        'entity': 'I-notation',
        'score': 0.97785944,
        'index': 31,  # This is moot for the purposes of this test.
        'word': '{',
        'start': 59,
        'end': 60},
    {
        'entity': 'I-notation',
        'score': 0.97785944,
        'index': 32,  # This is moot for the purposes of this test.
        'word': 'i',
        'start': 60,
        'end': 61},
    {
        'entity': 'I-notation',
        'score': 0.97785944,
        'index': 33,  # This is moot for the purposes of this test.
        'word': '}',
        'start': 61,
        'end': 62},
    {
        'entity': 'I-notation',
        'score': 0.97785944,
        'index': 34,  # This is moot for the purposes of this test.
        'word': '$',
        'start': 62,
        'end': 63},
    ])
test_eq(str(sample_output_2.tag), r'<span notation="" style="border-width:1px;border-style:solid;padding:3px">$\sqrt{I}$</span>')
# main_text.find('radical')


In [ ]:
#| hide

# With modernbert, the tokenizer tends to capture spaces, within tokens, which creates some subtle issues with spacing.
text = r'Definition 1.1. - We call $\mathscr{U}$-topos, or simply topos if there is no fear of confusion, a category $\mathrm{E}$ such that there is a site $\mathrm{C} \in \mathscr{U}$ such that $\mathrm{E}$ is equivalent to the category $\tilde{\mathrm{C}}$ of $\mathrm{U}$-sheaves of sets over $\mathrm{C}$ .'

part_1 = [
        {'entity': 'B-definition',
        'score': 0.9991014,
        'index': 9,
        'word': 'Ġ$\\',
        'start': 25,
        'end': 28},
        {'entity': 'I-definition',
        'score': 0.99974686,
        'index': 10,
        'word': 'mathscr',
        'start': 28,
        'end': 35},
        {'entity': 'I-definition',
        'score': 0.999652,
        'index': 11,
        'word': '{',
        'start': 35,
        'end': 36},
        {'entity': 'I-definition',
        'score': 0.99985695,
        'index': 12,
        'word': 'U',
        'start': 36,
        'end': 37},
        {'entity': 'I-definition',
        'score': 0.9997553,
        'index': 13,
        'word': '}$-',
        'start': 37,
        'end': 40},
        {'entity': 'I-definition',
        'score': 0.9997918,
        'index': 14,
        'word': 'top',
        'start': 40,
        'end': 43},
        {'entity': 'I-definition',
        'score': 0.9997335,
        'index': 15,
        'word': 'os',
        'start': 43,
        'end': 45},]

part_2 = [
        {'entity': 'B-definition',
        'score': 0.594063,
        'index': 19,
        'word': 'Ġto',
        'start': 56,
        'end': 59},
        {'entity': 'I-definition',
        'score': 0.9056522,
        'index': 20,
        'word': 'pos',
        'start': 59,
        'end': 62}]

sample_output_1: HTMLTagWithIndices = _html_tag_data_from_part(text, part_1)
assert not sample_output_1.tag.text.startswith(' ')
assert sample_output_1.tag.text == '$\\mathscr{U}$-topos'

sample_output_2: HTMLTagWithIndices = _html_tag_data_from_part(text, part_1)
assert not sample_output_2.tag.text.startswith(' ')
assert sample_output_2.tag.text == '$\\mathscr{U}$-topos'

In [ ]:
#| export
def _current_token_continues_the_previous_token(
        current_token: dict,
        previous_token: dict,
        note: Optional[VaultNote] = None,
        ) -> bool:
    """
    Helper function to `_divide_token_preds_into_parts`.
    """
    if current_token['entity'].startswith('I-'):
        if current_token['entity'][2:] == previous_token['entity'][2:]:
            return True
        elif note:
            warnings.warn(rf"""
                In the note {note.name} at {note.path()},
                The token '{previous_token['word']}' is marked as '{previous_token['entity']}'
                and the subsequent token '{current_token['word']}' is marked as '{current_token['entity']}',
                which is unusual because the two consecutive tokens seem to be of different
                entities, and yet the latter token does not start with a 'B-'.

                The latter token will be treated like the beginning of a new entity."""
                    )
            return False
    else:
        return False
        

In [ ]:
#| hide
previous_token_1 = {
        'entity': 'I-notation',
        'score': 0.97785944,
        'index': 33,  # This is moot for the purposes of this test.
        'word': '}',
        'start': 61,
        'end': 62}
current_token_1 = {
        'entity': 'I-notation',
        'score': 0.97785944,
        'index': 34,  # This is moot for the purposes of this test.
        'word': '$',
        'start': 62,
        'end': 63}
assert _current_token_continues_the_previous_token(current_token_1, previous_token_1, note=VaultNote('', rel_path='hi'))

# Something like below should hopefully not happen, but it should still give a warning message
previous_token_2 = {
        'entity': 'I-definition',
        'score': 0.97785944,
        'index': 33,  # This is moot for the purposes of this test.
        'word': '}',
        'start': 61,
        'end': 62}
current_token_2 = {
        'entity': 'I-notation',
        'score': 0.97785944,
        'index': 34,  # This is moot for the purposes of this test.
        'word': '$',
        'start': 62,
        'end': 63}

with warnings.catch_warnings(record=True) as w:
    sample_output = _current_token_continues_the_previous_token(current_token_2, previous_token_2, note=VaultNote('', rel_path='hi'))
    assert w
    assert not sample_output

previous_token_3 = {
        'entity': 'I-definition',
        'score': 0.97785944,
        'index': 33,  # This is moot for the purposes of this test.
        'word': '##tion',
        'start': 58,
        'end': 62}
current_token_3 = {
        'entity': 'B-notation',
        'score': 0.97785944,
        'index': 34,  # This is moot for the purposes of this test.
        'word': '$',
        'start': 62,
        'end': 63}

assert not _current_token_continues_the_previous_token(current_token_3, previous_token_3, note=VaultNote('', rel_path='hi'))



In [ ]:
#| export
def _divide_token_preds_into_parts(
        token_preds: list[dict[str]],
        excessive_space_threshold: int,
        note: Optional[VaultNote] = None
        ) -> list[list[dict[str]]]:
    """
    Divide `token_preds` into parts so that each part
    represents a single definition/notation marking.

    Helper function to `_html_tags_from_token_preds`.
    """
    token_preds_parts = []
    for current_token in token_preds:
        if not token_preds_parts:
            token_preds_parts.append([current_token])
            continue
        prev_token = token_preds_parts[-1][-1]
        if _current_token_continues_the_previous_token(
                current_token, prev_token, note):
            prev_token_end = prev_token['end']
            cur_token_start = current_token['start']
            if prev_token_end + excessive_space_threshold >= cur_token_start and note:
                Warning(rf"""
                    In the note {note.name} at {note.path()},
                    There seems to be excessive space between the token
                    {prev_token['word']} and {current_token['word']}, which
                    seem to be part of the same entity"""
                        )
            token_preds_parts[-1].append(current_token)
        else:
            token_preds_parts.append([current_token])
    return token_preds_parts


In [ ]:
#| hide

main_text = r"Let $I \subset A$ be an ideal. Define its radical by $\sqrt{I}$"

preds = [
    {
        'entity': 'B-definition',
        'score': 0.37319255,
        'index': 25,  # This is moot for the purposes of this test.
        'word': 'radical',
        'start': 42,
        'end': 49
    },
    {
        'entity': 'B-notation',
        'score': 0.67021805,
        'index': 27,  # This is moot for the purposes of this test.
        'word': '$',
        'start': 53,
        'end': 54},
    {
        'entity': 'I-notation',
        'score': 0.9748327,
        'index': 28,  # This is moot for the purposes of this test.
        'word': '\\',
        'start': 54,
        'end': 55},
    {
        'entity': 'I-notation',
        'score': 0.9754836,
        'index': 29,  # This is moot for the purposes of this test.
        'word': 'sq',
        'start': 55,
        'end': 57},
    {
        'entity': 'I-notation',
        'score': 0.9750675,
        'index': 30,  # This is moot for the purposes of this test.
        'word': '##rt',
        'start': 57,
        'end': 59},
    {
        'entity': 'I-notation',
        'score': 0.97785944,
        'index': 31,  # This is moot for the purposes of this test.
        'word': '{',
        'start': 59,
        'end': 60},
    {
        'entity': 'I-notation',
        'score': 0.97785944,
        'index': 32,  # This is moot for the purposes of this test.
        'word': 'i',
        'start': 60,
        'end': 61},
    {
        'entity': 'I-notation',
        'score': 0.97785944,
        'index': 33,  # This is moot for the purposes of this test.
        'word': '}',
        'start': 61,
        'end': 62},
    {
        'entity': 'I-notation',
        'score': 0.97785944,
        'index': 34,  # This is moot for the purposes of this test.
        'word': '$',
        'start': 62,
        'end': 63},
    ]

output = _divide_token_preds_into_parts(
    preds,  excessive_space_threshold=2, note=VaultNote('', rel_path='hi')
)

# Test that the list finds two parts, one for the definition, and the other for the notation.
test_eq(len(output), 2)
test_eq(len(output[0]), 1)
test_eq(len(output[1]), len(preds) - 1)


In [ ]:
#| export
def _ranges_overlap(
        current_1: HTMLTagWithIndices,
        current_2: HTMLTagWithIndices
        # current_1: tuple[bs4.element.Tag, int, int],
        # current_2: tuple[bs4.element.Tag, int, int]
        ) -> bool:
    """
    Based on https://stackoverflow.com/a/64745177

    Helper function to `_collate_html_tags`, `_consolidate_token_preds`.
    """
    return max(current_1.start, current_2.start) < min(current_1.end, current_2.end)


In [ ]:
#| hide
# In actuality, there should be bs4.element.Tag objects in place of ''.
assert _ranges_overlap(HTMLTagWithIndices('', 3, 8), HTMLTagWithIndices('', 6, 12))
assert _ranges_overlap(HTMLTagWithIndices('', 3, 8), HTMLTagWithIndices('', 3, 4))
assert not _ranges_overlap(HTMLTagWithIndices('', 3, 8), HTMLTagWithIndices('', 8, 9))
assert _ranges_overlap(HTMLTagWithIndices('', 6, 12), HTMLTagWithIndices('', 3, 8))

In [ ]:
#| export
# If the ML model predicts 
# predictions made around 
latex_commands_to_avoid = [
    # Document structure
    "\\documentclass", "\\usepackage", "\\begin", "\\end",
    # Sectioning
    "\\part", "\\chapter", "\\section", "\\subsection", "\\subsubsection", "\\paragraph", "\\subparagraph",
    # Referencing and citation
    "\\ref", "\\pageref", "\\cite", "\\bibitem", "\\bibliography", "\\bibliographystyle",
    
    # Lists
    "\\enumerate", "\\itemize", "\\item", "\\description",
    
    # Footnotes
    "\\footnote", "\\footnotemark", "\\footnotetext",
    
    # Floats and captions
    # "\\figure", "\\table", "\\caption", "\\centering",
    
    # Page formatting
    "\\newpage", "\\clearpage", "\\cleardoublepage", "\\pagebreak", "\\newline", "\\linebreak",
    
    # Definitions and counters
    "\\newcommand", "\\renewcommand", "\\def", "\\let", "\\setcounter", "\\addtocounter", "\\stepcounter", "\\refstepcounter",
    
    # Index and glossary
    "\\index", "\\glossary", "\\printindex", "\\printglossary",
    
    # Hyperlinks (if using hyperref package)
    "\\href", "\\url", "\\hypersetup", "\\hyperlink", "\\hypertarget",
    
    # Spacing
    "\\hspace", "\\vspace", "\\phantom", "\\vphantom", "\\hphantom",
    
    # Alignment environments
    "\\center", "\\flushleft", "\\flushright",
    
    # Verbatim and code
    "\\verb", "\\verbatim", "\\lstlisting",
    
    # Theorems and proofs
    "\\theorem", "\\lemma", "\\proof", "\\corollary",
    
    # Cross-referencing
    "\\cref", "\\Cref", "\\vref", "\\Vref",  # from cleveref package
    
    # Table of contents related
    "\\tableofcontents", "\\listoffigures", "\\listoftables",
    
    # Appendix
    "\\appendix",
    
    # Author and title information
    "\\title", "\\author", "\\date", "\\maketitle",
    
    # Abstract
    "\\abstract", "\\keywords",
    
    # Including external files
    "\\input", "\\include",
    
    # Page numbering
    "\\pagenumbering", "\\thepage",
    
    # Margin adjustments
    "\\marginpar", "\\marginparsep", "\\marginparwidth",
    
    # Columns
    "\\twocolumn", "\\onecolumn", "\\multicolumn",
]


In [ ]:
#| export
def _str_contains_latex_command_to_avoid(text):
    """
    Helper function to `_consolidate_token_preds`
    """
    # Check if any command from the list is in the text
    for command in latex_commands_to_avoid:
        if command.lower() in text:
            return True
    return False

In [ ]:
#| hide
test_strings = [
    r"This is a normal sentence.",
    r"This sentence contains \section{A Section}.",
    r"Here's a \footnote{This is a footnote.}",
    r"Let's \begin{enumerate} some items.",
    r"\href{https://example.com}{This is a hyperlink}",
]

for i, string in enumerate(test_strings, 1):
    result = _str_contains_latex_command_to_avoid(string)
    print(f"String {i}: {'Contains' if result else 'Does not contain'} a LaTeX command to avoid.")

String 1: Does not contain a LaTeX command to avoid.
String 2: Contains a LaTeX command to avoid.
String 3: Contains a LaTeX command to avoid.
String 4: Contains a LaTeX command to avoid.
String 5: Contains a LaTeX command to avoid.


In [ ]:
#| export
def _consolidate_token_preds(
        main_text: str,
        tag_data: list[HTMLTagWithIndices]
        ) -> list[HTMLTagWithIndices]:
    """
    Since the model's predictions can yield some odd results
    (e.g. notations not being marked for an entire LaTeX string
    $<span notation="">$S_k := ...</span>$$), this function tries
    to consolidate some oddities.
    
    """
    latex_inds = latex_indices(main_text)
    extended_tag_data = _extend_tag_data_ranges(main_text, latex_inds, tag_data)
    # extended_tag_data = _extend_tag_data_ranges_to_encompass_latex(main_text, latex_inds, tag_data)
    # extended_tag_data = _extend_tag_data_ranges_to_border_spaces(main_text, extended_tag_data)
    tag_data_notats_chopped = _cutoff_notation_tag_data(main_text, extended_tag_data)
    # Go through the extended tag data to throw out overlapping ones.
    ultimate_tag_data: list[HTMLTagWithIndices] = []
    for tag_point in tag_data_notats_chopped:
        if (_no_overlap_with_previous_tag_data(ultimate_tag_data, tag_point)
                and not _str_contains_latex_command_to_avoid(tag_point.tag.text)):
            ultimate_tag_data.append(tag_point)
    return ultimate_tag_data


def _extend_tag_data_ranges(
        main_text: str,
        latex_inds: list[tuple[int, int]],
        tag_data: list[HTMLTagWithIndices],
        ) -> list[HTMLTagWithIndices]:
    """
    Extend tag data so that
    1. the tag data does not start or end within any latex math mode string.
    2. the tag data is immediately preceded by whitespace (or the start/end of line)
       and followed by whitespace or punctuation
    3. the tag data does not start/end in the middle of the arguments of a latex command.

    Helper function to `_consolidate_token_preds`.
    """
    # TODO: make sure that tag has balanced curly braces 
    extended_tag_data = []
    for tag_tuple in tag_data:
        tag_tuple_before_extension = tag_tuple
        while True:
            tag_tuple_after_extension = _extend_tag_data_range_for_math_mode(
                tag_tuple_before_extension, main_text, latex_inds)
            tag_tuple_after_extension = _extend_tag_data_range_to_border_space_or_punc(
                tag_tuple_after_extension, main_text)
            tag_tuple_after_extension = _extend_tag_data_ranges_to_balance_curly_braces(
                tag_tuple_after_extension, main_text)
            tag_tuple_before_extension = tag_tuple_after_extension 
            if (tag_tuple_before_extension[1] == tag_tuple_after_extension[1]
                    and tag_tuple_before_extension[2] == tag_tuple_after_extension[2]):
                break
        extended_tag_data.append(tag_tuple_after_extension)
    return extended_tag_data


def _update_tag_data(
        tag_tuple: HTMLTagWithIndices,
        main_text: str,
        new_start: int,
        new_end: int) -> HTMLTagWithIndices:
    new_text = main_text[new_start:new_end]
    tag_type = 'definition' if 'definition' in tag_tuple.tag.attrs else 'notation'
    new_tag = _make_tag(new_text, tag_type)
    return HTMLTagWithIndices(new_tag, new_start, new_end)


def _extend_tag_data_range_for_math_mode(
        tag_tuple: HTMLTagWithIndices,
        main_text: str,
        latex_inds: list[tuple[int, int]],
        ) -> HTMLTagWithIndices:
    """
    Extend tag data so that the tag data does not start or end within latex math mode string.
    """
    extended_range = [tag_tuple.start, tag_tuple.end]
    for tex_range in latex_inds:
        if not _ranges_overlap(HTMLTagWithIndices(0, tex_range[0], tex_range[1]), tag_tuple):
            continue
        extended_range = (min(extended_range[0], tex_range[0]), max(extended_range[1], tex_range[1]))
    return _update_tag_data(tag_tuple, main_text, extended_range[0], extended_range[1])


def _extend_tag_data_range_to_border_space_or_punc(
        tag_tuple: HTMLTagWithIndices,
        main_text: str,
        ) -> HTMLTagWithIndices:
    """
    Extend tag data so that the tag data borders spaces or punctuations.

    Helper function to `_consolidate_token_preds`.
    """
    combined_range = [tag_tuple.start, tag_tuple.end]
    while combined_range[0] != 0 and not main_text[combined_range[0]-1].isspace():
        combined_range[0] -= 1
    # while combined_range[1] != len(main_text) and not main_text[combined_range[1]].isspace():
    while combined_range[1] != len(main_text) and is_not_space_and_not_punc(main_text[combined_range[1]]):
        combined_range[1] += 1
    return _update_tag_data(tag_tuple, main_text, combined_range[0], combined_range[1])


def _extend_tag_data_ranges_to_balance_curly_braces(
        tag_tuple: HTMLTagWithIndices,
        main_text: str
        ) -> list[HTMLTagWithIndices]:
    """
    Extend tag data to balance curly braces
    """
    combined_range = [tag_tuple.start, tag_tuple.end]
    while not _is_balanced_braces(main_text[combined_range[0]:combined_range[1]]):
        changed = False
        while combined_range[0] != 0 and _first_curly_bracket(main_text[combined_range[0]:combined_range[1]]) == r'}':
            combined_range[0] -= 1
            changed = True
        while combined_range[1] != len(main_text) and _last_curly_bracket(main_text[combined_range[0]:combined_range[1]]) == r'{':
            combined_range[1] += 1
            changed = True
        if not changed:
            break
    return _update_tag_data(tag_tuple, main_text, combined_range[0], combined_range[1])


def _cutoff_notation_tag_data(
        main_text: str,
        tag_data: list[HTMLTagWithIndices],
        ) -> list[HTMLTagWithIndices]:
    """
    Helper function to `_consolidate_token_preds`.

    Guarantees that a notation tag is a pure math mode latex string
    by cutting only the pure math mode string
    that occurs within it. Assumes that `_extend_tag_data_ranges_to_encompass_latex`
    works as intended.
    """
    cutout_notation_tag_data: list[HTMLTagWithIndices] = []
    for tag, start, end in tag_data:
        if not 'notation' in tag.attrs:
            cutout_notation_tag_data.append(HTMLTagWithIndices(tag, start, end))
            continue
        tag_text = main_text[start:end]
        tex_inds_in_tagged = latex_indices(tag_text)
        for sub_start, sub_end in tex_inds_in_tagged:
            tex_str = main_text[start+sub_start:start+sub_end]
            cutout_tag = _make_tag(tex_str, 'notation')
            cutout_notation_tag_data.append(
                HTMLTagWithIndices(cutout_tag, start+sub_start, start+sub_end))
    return cutout_notation_tag_data



def _no_overlap_with_previous_tag_data(
        ultimate_tag_data: list[HTMLTagWithIndices],
        current_tag_data: HTMLTagWithIndices  # Current tag data
        ) -> bool:
    """
    Return `True`, if the `current_tag_data` does not overlap with
    any tag data that will be ultimately added. 

    Helper function for `_consolidate_token_preds`.
    """
    for prev in reversed(ultimate_tag_data):
        if _ranges_overlap(prev, current_tag_data):
            return False
    return True
    

In [ ]:
#| hide
main_text = 'Hi. This is some text. Here is a notation $$M_k :=... $$ and here is some more $$G_k :=...$$'

soup = bs4.BeautifulSoup('', 'html.parser')
# In actuality, the tag data will have more information, but the following
# is good enough
# for the purposes of this test
tag_1 = soup.new_tag('span', notation='')
tag_2 = soup.new_tag('span', notation='')
# The following tries to test an erroneous short/incomplete marking of the very first 
# dollar sign `$` as well as the subtext 'G_K ' of the second math mode text.

tag_data = [HTMLTagWithIndices(tag_1, 42, 43), HTMLTagWithIndices(tag_2, 81, 85)]
output = _consolidate_token_preds(main_text, tag_data)
test_eq(main_text[output[0][1]: output[0][2]], '$$M_k :=... $$')
test_eq(main_text[output[1][1]: output[1][2]], '$$G_k :=...$$')


# In the following example, the notation is a priori
# found to be 'zeta(s)$ as $$\zeta'. 
# So first, the tagged data is extended to encompass
# '$\zeta(s)$ as $$\zeta(s) = ...$$' and then pure latex
# math mode str are extracted
main_text = r'Define $\zeta(s)$ as $$\zeta(s) = ...$$  '

soup = bs4.BeautifulSoup('', 'html.parser')
tag_1 = soup.new_tag('span', notation='')
tag_data = [HTMLTagWithIndices(tag_1,9,27)]
output = _consolidate_token_preds(main_text, tag_data)
# main_text.find('zeta')
test_eq(main_text[output[0][1]:output[0][2]], r'$\zeta(s)$')
test_eq(main_text[output[1][1]:output[1][2]], r'$$\zeta(s) = ...$$')



# In the following example, we have erroneous
# notation and definition markings which start/end within the same 
# math mode string.
# In this case, the preceding marking takes precedence and 
# the latter overlapping marking is discarded;
# this is an unfortunate feature that must be implemented to get
# around shortcomings of the model.
main_text = r'The Riemann zeta function $\zeta(s)$ is defined as...'
soup = bs4.BeautifulSoup('', 'html.parser')
tag_1 = soup.new_tag('b', definition='')
tag_2 = soup.new_tag('span', notation='')
tag_data = [HTMLTagWithIndices(tag_1,4,27), HTMLTagWithIndices(tag_2,26,30)] # the tag predictions overlap at $ \zeta(s)$.
output = _consolidate_token_preds(main_text, tag_data)
test_eq(main_text[output[0][1]:output[0][2]], r'Riemann zeta function $\zeta(s)$')

# In the following example, the definition tag prediction ends within the argument
# of the \emph command. 
main_text = r'The \emph{arc complex}'
soup = bs4.BeautifulSoup('', 'html.parser')
tag = soup.new_tag('b', definition="")
tag_data = [HTMLTagWithIndices(tag,4,21)]
output = _consolidate_token_preds(main_text, tag_data)
test_eq(main_text[output[0][1]:output[0][2]], r'\emph{arc complex}')

# In the following example, the definition tag prediction ends amidst of the \emph command. 
main_text = r'The \emph{arc complex}'
soup = bs4.BeautifulSoup('', 'html.parser')
tag = soup.new_tag('b', definition="")
tag_data = [HTMLTagWithIndices(tag,6,8)]
output = _consolidate_token_preds(main_text, tag_data)
test_eq(main_text[output[0][1]:output[0][2]], r'\emph{arc complex}')

# In the following example, a bad command gets predicted. 
# The consolidation should thus leave out this prediction.
main_text = r'\section{lalalala} We define a homotopy'
soup = bs4.BeautifulSoup('', 'html.parser')
tag = soup.new_tag('b', definition="")
tag_data = [HTMLTagWithIndices(tag,0,2)]
output = _consolidate_token_preds(main_text, tag_data)
test_eq(len(output), 0)
# print(output)

# In the following example, the definition tag prediction ends with a punctuation mark
# (a period in this case). 
main_text = r'This is called a homotopy. There exist...'
soup = bs4.BeautifulSoup('', 'html.parser')
tag = soup.new_tag('b', definition="")
tag_data = [HTMLTagWithIndices(tag,17,24)]
output = _consolidate_token_preds(main_text, tag_data)
test_eq(main_text[output[0][1]:output[0][2]], r'homotopy')

# The `_extend_tag_data_range_to_border_space_or_punc` function should not try to
# encapsulate the period as part of the definition tag.
tag_data = [HTMLTagWithIndices(tag,18,23)]
output = _consolidate_token_preds(main_text, tag_data)
test_eq(main_text[output[0][1]:output[0][2]], r'homotopy')


In [ ]:
#| export
def _html_tags_from_token_preds(
        main_text: str,
        token_preds: list[dict[str]], # An output of `pipeline(text)`; Each dict likely contains keys such as `'entity'`, `'score'`, `'index'`, `'word'`, `'start'`, and `'end'`, depending on the model used.
        excessive_space_threshold: int,
        note: Optional[VaultNote] = None,
        ) -> list[HTMLTagWithIndices]:  # Tag element, start, end, where main_text[start:end] needs to be replaced by the tag element.
        # ) -> list[tuple[bs4.element.Tag, int, int]]:  # Tag element, start, end, where main_text[start:end] needs to be replaced by the tag element.
    """
    Return HTML tags for definition and notation classification.

    Helper function to `auto_mark_def_and_notats`.
    """
    parts: list[list[dict[str]]] = _divide_token_preds_into_parts(
        token_preds, excessive_space_threshold, note)
    return [_html_tag_data_from_part(main_text, part) for part in parts]


In [ ]:
#| hide
# TODO: test _html_tags_from_token_preds

In [ ]:
#| export
def _collate_html_tags(
        tag_data_1: list[HTMLTagWithIndices],
        tag_data_2: list[HTMLTagWithIndices],
    ) -> list[tuple[bs4.element.Tag], int, int]:
    """
    Collates the lists of HTML tags and the indices within a certain text
    (which is not-needed for this function and hence not included)
    that the HTML tags need to replace.

    If there are entries in `tag_data_1` and `tag_data_2` with overlapping
    ranges, then the entry from `tag_data_1` is prioritized and the entry
    from `tag_data_2` is discarded.

    Helper function to `auto_mark_def_and_notats`
    """
    collated_list: list[HTMLTagWithIndices] = []
    i, j = 0, 0
    while i < len(tag_data_1) and j < len(tag_data_2):
        current_1 = tag_data_1[i]
        current_2 = tag_data_2[j]
        if _ranges_overlap(current_1, current_2): # Ignore current_2
            j += 1
            continue
        if current_1[1] > current_2[1]:
            collated_list.append(current_2)
            j += 1
        else:
            collated_list.append(current_1)
            i += 1
    while i < len(tag_data_1):
        collated_list.append(tag_data_1[i])
        i += 1
    while j < len(tag_data_2):
        collated_list.append(tag_data_2[j])
        j += 1
    return collated_list




In [ ]:
#| hide


tag_data_1 = [
    HTMLTagWithIndices('', 0, 1),
    HTMLTagWithIndices('', 9, 12),
    HTMLTagWithIndices('', 20, 21)
]

tag_data_2 = [
    HTMLTagWithIndices('', 2, 4),
    HTMLTagWithIndices('', 6, 7),
    HTMLTagWithIndices('', 8, 10), # This should be discarded
    HTMLTagWithIndices('', 10, 13), # This should be discarded
    HTMLTagWithIndices('', 17, 20),
    HTMLTagWithIndices('', 21, 24)
]
output = _collate_html_tags(tag_data_1, tag_data_2)
test_eq(output, [
    HTMLTagWithIndices('', 0, 1), HTMLTagWithIndices('', 2, 4), HTMLTagWithIndices('', 6, 7),
    HTMLTagWithIndices('', 9, 12), HTMLTagWithIndices('', 17, 20), HTMLTagWithIndices('', 20, 21), 
    HTMLTagWithIndices('', 21, 24)])

In [ ]:
#| export
def _add_nice_boxing_attrs_to_def_and_notat_tags(
        html_tag_data: list[HTMLTagWithIndices]
        ) -> list[HTMLTagWithIndices]:
    """
    Add HTML tag attributes to draw boxes around notation data

    Helper function to `auto_mark_def_and_notats`.
    """
    listy: list[HTMLTagWithIndices] = []
    for tag, start, end in html_tag_data:
        if ('notation' in tag.attrs or 'definition' in tag.attrs) and 'style' not in tag.attrs:
            tag.attrs['style'] = "border-width:1px;border-style:solid;padding:3px"
        listy.append(HTMLTagWithIndices(tag, start, end)) 
    return listy



In [ ]:
#| hide
soup = bs4.BeautifulSoup('', "html.parser")
tag = soup.new_tag("span", notation="")
tag.string = 'hi'
tag_data = [
    HTMLTagWithIndices(tag, 0, 2),
]
output = _add_nice_boxing_attrs_to_def_and_notat_tags(tag_data)
assert "style" in output[0][0].attrs

tag = soup.new_tag("span", definition="")
tag.string = 'hi'
tag_data = [
    HTMLTagWithIndices(tag, 0, 2),
]
output = _add_nice_boxing_attrs_to_def_and_notat_tags(tag_data)
assert "style" in output[0][0].attrs

In [ ]:
#| export
def def_and_notat_preds_by_model(
        text: str,  
        pipeline # The pipeline object created using the token classification model and its tokenizer
        ) -> list[HTMLTagWithIndices]: # HTMLTAgWithIndices consists of an HTML tag carrying the data of the prediction and ints marking where in `text` the definition or notation is at.
    """
    Predict where definitions and notations occur in `text`

    This function uses some of the same helper functions as
    `auto_mark_def_and_notats`, but does not raise warning messages as
    in `auto_mark_def_and_notats`.
    """
    tag_data = _html_tags_from_token_preds(text, pipeline(text), 2, None)
    return tag_data

In [ ]:
# TODO: test

In [ ]:
#| export
# def _process_mf(
#         mf: MarkdownFile) -> None:
#     """
#     Merge display math mode as needed

#     Helper function to `auto_mark_def_and_notats`
#     """
#     # mf.merge_display_math_mode()
#     # mf.merge_display_math_mode_into_preceding_text()
#     mf.

In [ ]:
#| export
def _get_main_text_lines(
        mf: MarkdownFile) -> tuple[int, int]:
    """Helper function to `auto_mark_def_and_notats`"""
    tuppy = mf.metadata_lines()
    if tuppy is not None:
        first_non_metadata_line = tuppy[1] + 1
    else:
        first_non_metadata_line = 0 
    see_also_line = mf.get_line_number_of_heading('See Also')
    return first_non_metadata_line, see_also_line



In [ ]:
#| export
def _append_to_pieces_start_and_end(
        pieces_start_and_end: list[tuple[int, int]],
        start_chunk: tuple[str, int, int],
        end_chunk: tuple[str, int, int]
        ) -> None:
    """Helper function to `_find_places_to_divide_from_chunks`"""
    start_char_index = start_chunk[1]
    end_char_index = end_chunk[1] + len(end_chunk[0])
    pieces_start_and_end.append([start_char_index, end_char_index])

In [ ]:
#| export
def _find_places_to_divide_from_chunks(
        chunks: list[tuple[str, int, int]], # The str is a chunk of text, the first int is the index in `main_text` that the chunk starts at, and the second int is the approximate token length of the text. Appending all the chunks of text as they are should result back in the original text.
        pipeline: pipelines.token_classification.TokenClassificationPipeline, # The token classification pipeline that is used to predict whether tokens are part of definitions or notations introduced in the text. Here, the tokenizer of this pipeline is used to estimate how many tokens a piece of subtext will have.
        ) -> list[tuple[int, int]]: # Each tuple is a start and end range for pieces of `main_text` to be considered for predictions
    """Identify appropriate indices in `main_text` where (overlapping)
    pieces in `main_text` should start/end for predictions with `pipeline`.
    
    Helper function to `_divide_main_text`.

    We describe how this function is implemented: starting at the first chunk
    (chunks are non-overlapping), start to consider consecutive chunks to
    make up a piece. So maybe we have chunks

        A B C D E F ....

    We build a piece chunk-by-chunk, considering the total token length of the
    built sub-piece along the way. The first chunk within a sub-piece 
    that makes the sub-piece of token-length greater than half the max
    token length with respect to `pipeline.tokenizer` will become the start of the
    next piece, unless the very first chunk in the piece is already longer than half the max
    token length with respect to the tokenizer (this is to ensure that the
    piece-building process does not keep starting at the same chunk).
    Moreover, a piece will stop building as soon as its token-length exceeds
    the max length of the tokenizer.

    For instance, maybe the max length (`tokenizer.model_max_length`)
    for the tokenizer is 512, and the chunks
    are of the following length:

        A   B    C  D   E    F     ...
        76  130  70 13  150  140   ...

    We first build the piece starting at A:

        A
        76

    We continue building the piece by "appending" B:

        A   B
        76  130

    Once we append C as well, the piece's length is now 276 and hence over half of 512,
    so the next piece will start at C: 

        A   B    C
        76  130  70

    Subsequently, we continue building the piece. Only once F is appended does the 
    length of the entire piece exceed 512 (the length is 579):

        A   B    C  D   E    F
        76  130  70 13  150  140
    
    And then we begin building the next piece from C.

    Also, consider an example where the first chunk's length exceeds half the max length
    of the tokenizer:

        A   B    C   ...
        300 200  100 ...

    Here, the first piece will consist of the chunks A, B, and C because
    the length of the piece exceeds the max length of 512 only after appending C.
    To guarantee that the next piece does not start with the chunk A again, B is 
    used as the first chunk in the next piece:

        B    C   ...
        200  100 ...

    If any chunk's token length exceeds the tokenizer's max_model_length, then
    the pipeline/model can only predict on the starting tokens in the chunk. 
    As such, the chunks must not be "too long" for best results on the model's predictions.
    """
    tokenizer = pipeline.tokenizer
    start_chunk_index, next_piece_start_chunk_index = 0, 0
    current_piece_token_len = 0
    pieces_start_and_end = []
    i = 0
    while i < len(chunks):
        chunk = chunks[i]
        current_piece_token_len = current_piece_token_len + chunk[2]
        if (current_piece_token_len > tokenizer.model_max_length / 2
                and start_chunk_index == next_piece_start_chunk_index):
            # Mark where the next piece should start
            next_piece_start_chunk_index = i if start_chunk_index != i else i+1
        if (current_piece_token_len > tokenizer.model_max_length):
            _append_to_pieces_start_and_end(
                pieces_start_and_end, chunks[start_chunk_index], chunk)
            i, start_chunk_index = (
                next_piece_start_chunk_index, next_piece_start_chunk_index)
            current_piece_token_len = 0
            continue
        i += 1
    # Add the last chunk at the end
    _append_to_pieces_start_and_end(
        pieces_start_and_end, chunks[start_chunk_index], chunks[-1])
    return pieces_start_and_end



In [ ]:
#| export
def _divide_main_text(
        main_text: str,
        pipeline: pipelines.token_classification.TokenClassificationPipeline, # The token classification pipeline that is used to predict whether tokens are part of definitions or notations introduced in the text. Here, the tokenizer of this pipeline is used to estimate how many tokens a piece of subtext will have.
        # ) -> list[tuple[str, int, int]]:  # The str is a chunk of text, the first int is the index in `main_text` that the chunk starts at, and the second int is the approximate token length of the text. Appending all the chunks of text as they are should result back in the original text.
        ) -> list[tuple[int, int]]:  # Each tuple is a start and end range for pieces of `main_text` to be considered for predictions
    """Divides `main_text` so that predictions can be made on smaller chunks of text.
    
    Assumes that dividing `main_text` along newline characters `\n` will result in
    pieces that are "not too long".

    Helper function to `_format_main_text_and_add_html_tag_data`.


    """
    main_text.split('\n')
    tokenizer = pipeline.tokenizer
    newline_indices = [i for i, char in enumerate(main_text) if char == '\n']
    newline_indices.insert(0, 0)
    chunks = []  # list[tuple[str, int, int]]  # The str is a chunk of text, the first int is the index in `main_text` that the chunk starts at, and the second int is the approximate token length of the text. Appending all the chunks of text as they are should result back in the original text.
    for start, end in pairwise(newline_indices):
        chunk = main_text[start:end]
        chunks.append((chunk, start, len(tokenizer(chunk)['input_ids'])))
    last_chunk = main_text[newline_indices[-1]:]
    chunks.append((last_chunk, newline_indices[-1], len(tokenizer(last_chunk)['input_ids'])))
    return _find_places_to_divide_from_chunks(chunks, pipeline)

In [ ]:
#| export
def _get_token_preds_by_dividing_main_text(
        main_text: str,
        pipeline: pipelines.token_classification.TokenClassificationPipeline, # The token classification pipeline that is used to predict whether tokens are part of definitions or notations introduced in the text. Here, the tokenizer of this pipeline is used to estimate how many tokens a piece of subtext will have.
        excessive_space_threshold: int, 
        note: Optional[VaultNote] = None,
        # ) -> list[tuple[bs4.element.Tag, int, int]]:  # Tag element, start, end, where main_text[start:end] needs to be replaced by the tag element.
        ) -> list[HTMLTagWithIndices]:  # Tag element, start, end, where main_text[start:end] needs to be replaced by the tag element.
    """
    Divide the `main_text` into not-too-long pieces to return HTML tag predictions

    Helper function for `_format_main_text_and_add_html_tag_data`.
    """
    pieces_start_and_end = _divide_main_text(main_text, pipeline)
    cumulative_html_tags_in_main = []
    for start_of_piece, end_of_piece in pieces_start_and_end:
        # text = main_text[start_of_piece:end_of_piece]
        text = main_text[start_of_piece:]
        html_tags_in_piece: list[HTMLTagWithIndices] = _html_tags_from_token_preds(
            text, pipeline(text), excessive_space_threshold, note)
        html_tags_in_piece = _consolidate_token_preds(
            text, html_tags_in_piece)
        # start and end indices need to be re-adjusted with respect to their places in `main_text`
        html_tags_for_piece_in_main_text: list[HTMLTagWithIndices] = [
            HTMLTagWithIndices(tag, start_of_piece + start, start_of_piece + end)
            for tag, start, end in html_tags_in_piece]
        cumulative_html_tags_in_main = _collate_html_tags(
            cumulative_html_tags_in_main, html_tags_for_piece_in_main_text)
    return cumulative_html_tags_in_main


In [ ]:
#| export
def get_def_and_notat_predictions(
        main_text: str,
        pipeline: pipelines.token_classification.TokenClassificationPipeline,
        excessive_space_threshold: int,
        note: Optional[VaultNote] = None
        ) -> list[HTMLTagWithIndices]:
    """
    Identifies definitions and notations in the text using the ML pipeline.

    Returns a list of `HTMLTagWithIndices` containing the predicted tags 
    and their start/end indices in `main_text`.
    """
    # 1. Divide text (if needed) and get raw token predictions
    html_tags_to_add = _get_token_preds_by_dividing_main_text(
        main_text, pipeline, excessive_space_threshold, note)
    
    # 2. Return the structured data directly
    # Note: _get_token_preds_by_dividing_main_text already returns 
    # list[HTMLTagWithIndices] after calling `consolidate_token_preds`.
    return html_tags_to_add


In [ ]:
#| export
#| export
def mark_def_and_notat_predictions(
        main_text: str, # The original text.
        predictions: list[HTMLTagWithIndices], # The list of predictions (ranges and metadata) returned by `get_def_and_notat_predictions`. Assumes these are sorted and non-overlapping (which `get_def_and_notat_predictions` ensures).
        formatter: Callable[[str, HTMLTagWithIndices], str] # A function that takes the *extracted substring* (the text being marked) and the prediction object, and returns the *formatted string* to replace it with.
        ) -> str: # The modified text with predictions marked.
    """
    Applies formatting to the `main_text` based on the definition and notation predictions.
    """
    # Iterate backwards so that index changes don't affect subsequent replacements
    # (HTMLTagWithIndices are typically sorted by start index, so reversing is safe)
    sorted_preds = sorted(predictions, key=lambda x: x.start, reverse=True)
    
    formatted_text_list = list(main_text)
    
    for pred in sorted_preds:
        start, end = pred.start, pred.end
        original_substring = main_text[start:end]
        
        # Apply the user-defined formatting rule
        replacement_text = formatter(original_substring, pred)
        
        # Replace the slice in the text
        # (Using list slicing for efficiency vs repeated string concatenation)
        formatted_text_list[start:end] = list(replacement_text)
        
    return "".join(formatted_text_list)


In [ ]:
from bs4 import BeautifulSoup
# String: "The Galois group $\operatorname{Gal}(L/K)$ is..."
# Indices breakdown:
# "The Galois group " -> len 17 (indices 0-16)
# "$" -> index 17
# "\operatorname{Gal}(L/K)" -> len 23
# "$" -> index 17 + 1 + 23 = 41
# String slice [17:42] covers "$\operatorname{Gal}(L/K)$"
text = r"The Galois group $\operatorname{Gal}(L/K)$ is..."
soup = BeautifulSoup("", 'html.parser')

# Pred 1: Definition "Galois group" (indices 4-16)
tag_def = soup.new_tag("b", definition="")
pred_def = HTMLTagWithIndices(tag_def, 4, 16)

# Pred 2: Notation "$\operatorname{Gal}(L/K)$"
# We want to wrap the WHOLE math string including $.
# Start: 17
# End: 42 (17 + 25 chars)
tag_notat = soup.new_tag("span", notation="")
pred_notat = HTMLTagWithIndices(tag_notat, 17, 42)

predictions = [pred_def, pred_notat]

def bracket_formatter(subtext, pred):
    label = "DEF" if 'definition' in pred.tag.attrs else "NOT"
    return f"[{label}:{subtext}]"

result = mark_def_and_notat_predictions(text, predictions, bracket_formatter)

# Expected: "The [DEF:Galois group] [NOT:$\operatorname{Gal}(L/K)$] is..."
expected_str = r"The [DEF:Galois group] [NOT:$\operatorname{Gal}(L/K)$] is..."

assert result == expected_str, f"Test 1 Failed.\nExpected: {expected_str}\nGot:      {result}"
print("Test 1 Passed:", result)

# --- Test 2: HTML Formatter ---
# Checks that we can regenerate standard HTML tags
def simple_html_formatter(subtext, pred):
    # Important: Clone the tag if you want to be pure, 
    # but modifying pred.tag is usually fine for one-pass formatting.
    pred.tag.string = subtext
    return str(pred.tag)

result_html = mark_def_and_notat_predictions(text, predictions, simple_html_formatter)

# Expected: "The <b definition="">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span> is..."
expected_html_fragment_1 = r'<b definition="">Galois group</b>'
expected_html_fragment_2 = r'<span notation="">$\operatorname{Gal}(L/K)$</span>'

assert expected_html_fragment_1 in result_html
assert expected_html_fragment_2 in result_html
print("Test 2 Passed:", result_html)


Test 1 Passed: The [DEF:Galois group] [NOT:$\operatorname{Gal}(L/K)$] is...
Test 2 Passed: The <b definition="">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span> is...


In [ ]:
from unittest.mock import MagicMock

class MockTokenizer:
    """Mocks a HuggingFace tokenizer."""
    model_max_length = 512
    
    def __call__(self, text):
        # CORRECTED: Return a dict, not SimpleNamespace
        length = len(text) // 5 + 1
        return {'input_ids': [0] * length}

# Update the pipeline to use the corrected tokenizer
class MockPipeline:
    """Mocks the TokenClassificationPipeline."""
    def __init__(self, preds):
        self.tokenizer = MockTokenizer()
        self._preds = preds

    def __call__(self, text):
        return self._preds


In [ ]:

#| hide
from types import SimpleNamespace

# 1. Setup Mock Note
mock_note = SimpleNamespace(name="Test Note", path=lambda: "test/path/Test Note.md") 

# 2. Setup Input Text
# "The Galois group $\operatorname{Gal}(L/K)$ of the extension L/K is..."
# Indices:
# T=0, h=1, e=2,  =3
# G=4, a=5, l=6, o=7, i=8, s=9,  =10
# g=11, r=12, o=13, u=14, p=15
#  =16
# $=17, \=18, o=19, p=20 ...
text_input = r"The Galois group $\operatorname{Gal}(L/K)$ of the extension L/K is..."

# 3. Setup Mock Predictions
# We simulate the model finding:
# A. 'Galois group' as a definition.
# B. 'Gal' (inside the latex) as a notation.
mock_preds = [
    # Definition: "Galois group" (indices 4-16)
    {
        'entity': 'B-definition', 'score': 0.99, 'index': 1, 
        'word': 'Galois', 'start': 4, 'end': 10
    },
    {
        'entity': 'I-definition', 'score': 0.99, 'index': 2, 
        'word': 'group', 'start': 11, 'end': 16
    },
    # Notation: Model finds 'Gal' inside '$\operatorname{Gal}(L/K)$'
    # '$\operatorname{Gal}(L/K)$' starts at 17.
    # $ = 17
    # \operatorname{ = 18 to 31 (len 13)
    # Gal = 31 to 34
    {
        'entity': 'B-notation', 'score': 0.98, 'index': 3, 
        'word': 'Gal', 'start': 31, 'end': 34
    }
]

mock_pipeline = MockPipeline(mock_preds)

# 4. Run Function
preds = get_def_and_notat_predictions(
    main_text=text_input,
    pipeline=mock_pipeline,
    excessive_space_threshold=2,
    note=mock_note,
)

# 5. Assertions

# Verify we got 2 items
assert len(preds) == 2, f"Expected 2 predictions, got {len(preds)}"

# --- Check Definition ("Galois group") ---
# The function might return them in order of appearance. 
# 'Galois group' starts at 4, '$\operatorname...$' starts at 17.
def_pred = preds[0]
assert def_pred.tag.text.strip() == 'Galois group'
assert 'definition' in def_pred.tag.attrs
assert def_pred.start == 4
assert def_pred.end == 16

# --- Check Notation ("$\operatorname{Gal}(L/K)$") ---
# The model predicted 'Gal' (31-34).
# The logic should assume 'Gal' implies the surrounding math mode string is the notation.
# The full math string is `$\operatorname{Gal}(L/K)$`.
# Start index: 17 ($)
# End index: 17 + len('$\operatorname{Gal}(L/K)$') = 17 + 25 = 42
notat_pred = preds[1]
assert 'notation' in notat_pred.tag.attrs
assert notat_pred.start == 17
assert notat_pred.end == 42 # Check length of string manually if this fails
assert notat_pred.tag.text == r'$\operatorname{Gal}(L/K)$'


In [ ]:
#| export
def predict_and_mark_def_and_notats(
        main_text: str, # The text to run predictions on and format.
        pipeline: pipelines.token_classification.TokenClassificationPipeline, # The token classification pipeline.
        # note: VaultNote, # The note associated with the text (used for debugging/logging).
        formatter: Callable[[str, HTMLTagWithIndices], str], # The function that formats the text based on predictions.
        excessive_space_threshold: int # Threshold for detecting excessive spacing in predictions.
        ) -> str: # The formatted text.
    """
    Runs definition and notation detection on `main_text` and applies the `formatter`.
    """
    predictions = get_def_and_notat_predictions(
        main_text, pipeline, excessive_space_threshold)
    return mark_def_and_notat_predictions(main_text, predictions, formatter)


In [ ]:
from bs4 import BeautifulSoup
from unittest.mock import MagicMock
from fastcore.test import test_eq # Import fastcore's test_eq

# --- Setup ---
text_input = r"The Galois group $\operatorname{Gal}(L/K)$ is..."

mock_preds = [
    {'entity': 'B-definition', 'score': 0.99, 'index': 1, 'word': 'Galois', 'start': 4, 'end': 10},
    {'entity': 'I-definition', 'score': 0.99, 'index': 2, 'word': 'group', 'start': 11, 'end': 16},
    {'entity': 'B-notation', 'score': 0.98, 'index': 3, 'word': 'Gal', 'start': 31, 'end': 34}
]
mock_pipeline = MockPipeline(mock_preds)

# mock_note = MagicMock()
# mock_note.name = "Test Note"
# mock_note.path.return_value = "test_note.md"

def html_formatter(subtext, pred):
    pred.tag.string = subtext
    return str(pred.tag)

# --- Run Function ---
result_html = predict_and_mark_def_and_notats(
    main_text=text_input,
    pipeline=mock_pipeline,
    formatter=html_formatter,
    excessive_space_threshold=2
)

# --- Define Expectations ---
# Based on the output you saw earlier, we define the exact expected string.
# We include the specific style attributes that seem to be generated by default in your environment.
expected_html = (
    r'The <b definition="" style="border-width:1px;border-style:solid;padding:3px">Galois group</b> '
    r'<span notation="" style="border-width:1px;border-style:solid;padding:3px">$\operatorname{Gal}(L/K)$</span> is...'
)

# --- Assertion with fastcore ---
# test_eq will raise an error if they differ and print:
# "test_eq: expected vs actual" followed by the two strings.
# If they are long, it usually helps spot the difference.
test_eq(result_html, expected_html)

print("Test 1 (HTML) Passed with fastcore!")


# --- Test 2: Bracket Formatter (Debug/Sanity Check) ---
def bracket_formatter(subtext, pred):
    tag_type = "DEF" if 'definition' in pred.tag.attrs else "NOT"
    return f"[{tag_type}:{subtext}]"

result_bracket = predict_and_mark_def_and_notats(
    main_text=text_input,
    pipeline=mock_pipeline,
    # note=mock_note,
    formatter=bracket_formatter,
    excessive_space_threshold=2
)

expected_bracket = r"The [DEF:Galois group] [NOT:$\operatorname{Gal}(L/K)$] is..."
assert result_bracket == expected_bracket, f"Bracket test failed. Got: {result_bracket}"
print("Test 2 (Bracket) Passed!")


Test 1 (HTML) Passed with fastcore!
Test 2 (Bracket) Passed!


In [ ]:
#| export
def _format_main_text_and_add_html_tag_data(
        note: VaultNote,
        pipeline: pipelines.token_classification.TokenClassificationPipeline, # The token classification pipeline that is used to predict whether tokens are part of definitions or notations introduced in the text.
        add_boxing_attr_to_existing_def_and_notat_markings: bool,
        excessive_space_threshold: int,
        main_text: str,  # The main text to format and to add HTML tag data to
        ) -> str:
    """Helper function to `auto_mark_def_and_notats`"""
    # 1. Pre-processing
    main_text = add_space_to_lt_symbols_without_space(main_text)
    main_text = convert_double_asterisks_to_html_tags(main_text)
    main_text, existing_html_tag_data = remove_html_tags_in_text(main_text)

    # 2. Handle existing tag styling
    if add_boxing_attr_to_existing_def_and_notat_markings:
        existing_html_tag_data = _add_nice_boxing_attrs_to_def_and_notat_tags(
            existing_html_tag_data)

    # 3. Get New Predictions
    new_tag_data = get_def_and_notat_predictions(
        main_text, pipeline, excessive_space_threshold, note)

    # 4. Merge old and new tags
    all_tags_to_add = _collate_html_tags(
        existing_html_tag_data, new_tag_data)

    # 5. Apply to text
    return add_HTML_tag_data_to_raw_text(main_text, all_tags_to_add)
    # html_tags_to_add = _get_token_preds_by_dividing_main_text(
    #     main_text, pipeline, note, excessive_space_threshold)

    # html_tags_to_add_back = _collate_html_tags(
    #     existing_html_tag_data, html_tags_to_add)
    # return add_HTML_tag_data_to_raw_text(main_text, html_tags_to_add_back)



In [ ]:

#| hide
#| notest
from types import SimpleNamespace
from unittest.mock import MagicMock

# --- Test Setup ---

# 1. Setup a Mock Note
# Use MagicMock so .path() is callable if needed, or .path works as a property
mock_note = MagicMock()
mock_note.name = "Test Note"
mock_note.path.return_value = "test/path/Test Note.md" 

# 2. Setup Input Text
# "The Galois group $\operatorname{Gal}(L/K)$ of the extension L/K is..."
# Indices:
# T=0, h=1, e=2,  =3
# G=4, a=5, l=6, o=7, i=8, s=9,  =10 (Galois)
# g=11, r=12, o=13, u=14, p=15 (group)
#  =16
# $=17, \=18 ... $\operatorname{Gal}(L/K)$ ends at 42
text_input = r"The Galois group $\operatorname{Gal}(L/K)$ of the extension L/K is..."

# 3. Setup Mock Predictions
# We simulate the model finding:
# A. 'Galois group' as a definition.
# B. 'Gal' (inside the latex) as a notation.
mock_preds = [
    # Definition: "Galois group" (indices 4-16)
    {
        'entity': 'B-definition', 'score': 0.99, 'index': 1, 
        'word': 'Galois', 'start': 4, 'end': 10
    },
    {
        'entity': 'I-definition', 'score': 0.99, 'index': 2, 
        'word': 'group', 'start': 11, 'end': 16
    },
    # Notation: Model finds 'Gal' inside '$\operatorname{Gal}(L/K)$'
    # We simulate it finding the substring at index 31-34 (inside the operatorname)
    {
        'entity': 'B-notation', 'score': 0.98, 'index': 3, 
        'word': 'Gal', 'start': 31, 'end': 34
    }
]

mock_pipeline = MockPipeline(mock_preds)

# --- Run Function ---
formatted_text = _format_main_text_and_add_html_tag_data(
    note=mock_note,
    pipeline=mock_pipeline,
    add_boxing_attr_to_existing_def_and_notat_markings=True,
    excessive_space_threshold=2,
    main_text=text_input
)

# --- Assertions ---

# 1. Check if "Galois group" is wrapped in a definition tag
# It should be boxed because we set add_boxing...=True
# Note: exact tag name (b vs span) depends on your implementation.
# We check for the key components.
assert 'Galois group' in formatted_text
assert 'definition=""' in formatted_text
assert 'style="border-width:1px;border-style:solid;padding:3px"' in formatted_text

# 2. Check if $\operatorname{Gal}(L/K)$ is wrapped in a notation tag
# The logic should have expanded 'Gal' to the full math string.
expected_notation_str = r'$\operatorname{Gal}(L/K)$'
expected_notation_html = f'<span notation="" style="border-width:1px;border-style:solid;padding:3px">{expected_notation_str}</span>'

assert expected_notation_html in formatted_text, \
    f"Notation tag missing or incorrect.\nExpected: {expected_notation_html}\nGot: {formatted_text}"

# --- Test Case 2: Handling Existing Tags ---
# Text with existing "Extension field L" definition
text_with_existing = r'The <b definition="">extension field $L$</b> is...'
mock_pipeline_empty = MockPipeline([]) 

formatted_text_2 = _format_main_text_and_add_html_tag_data(
    note=mock_note,
    pipeline=mock_pipeline_empty,
    add_boxing_attr_to_existing_def_and_notat_markings=True, 
    excessive_space_threshold=2,
    main_text=text_with_existing
)

# Check if the 'style' attribute was added (boxing) to the existing bold tag
assert 'style="border-width:1px;border-style:solid;padding:3px"' in formatted_text_2, \
    "Boxing attributes were not added to existing tag."
assert '<b definition="" style=' in formatted_text_2

# --- Test Case 3: Double Asterisks Conversion ---
# The function calls `convert_double_asterisks_to_html_tags` internally.
# Example: **$K$**
text_asterisks = r"Let **$K$** be a field."
formatted_text_3 = _format_main_text_and_add_html_tag_data(
    note=mock_note,
    pipeline=mock_pipeline_empty,
    add_boxing_attr_to_existing_def_and_notat_markings=False,
    excessive_space_threshold=2,
    main_text=text_asterisks
)

# Should convert **...** to a tag (definition or notation depending on content/heuristics)
# Usually math mode inside ** -> notation.
assert '<span notation="">$K$</span>' in formatted_text_3 or '<b definition="">$K$</b>' in formatted_text_3


C:\Users\hyunj\Documents\Development\Python\trouver\trouver\helper\html.py:104: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  parsed_soup = BeautifulSoup(text, 'html.parser')


In [ ]:
#| export
def _write_text_with_html_tag_preds_to_note(
        note: VaultNote,
        mf: MarkdownFile,
        main_text: str,
        first_non_metadata_line: int,
        see_also_line: int
        ) -> None:
    """
    Final step of the marking process; the new contents of the note are written.

    Helper function to `auto_mark_def_and_notats`
    """
    mf.remove_lines(first_non_metadata_line, see_also_line)
    mf.insert_line(first_non_metadata_line,
                   {'type': MarkdownLineEnum.DEFAULT, 'line': main_text})
    mf.add_tags('_auto/def_and_notat_identified')
    mf.write(note)

In [ ]:
#| export
def auto_mark_def_and_notats(
        note: VaultNote,  # The standard information note in which to find the definitions and notations.
        pipeline: pipelines.token_classification.TokenClassificationPipeline, # The token classification pipeline that is used to predict whether tokens are part of definitions or notations introduced in the text.
        # remove_existing_def_and_notat_markings: bool = False,  # If `True`, remove definition and notation markings (both via surrounding by double asterisks `**` as per the legacy method and via HTML tags)
        excessive_space_threshold: int = 2,
        add_boxing_attr_to_existing_def_and_notat_markings: bool = True # If `True`, then nice attributes are added to the existing notation HTML tags, if not already present.
    ) -> None:
    """
    Predict and mark where definitions and notation occur in a note using
    a token classification ML model.

    Assumes that the note is a standard information note that does not
    have a lot of "user modifications", such as footnotes, links,
    and HTML tags. If
    there are many modifications, then these might be deleted.

    Assumes that the paragraphs in the text of the note are "not too long".
    Currently, this means that the paragraphs in the number of tokens
    in the text of the note should (roughly) not exceed 
    `pipeline.tokenizer.model_max_length`.

    Existing markings for definition and notation data (i.e. by
    surrounding with double asterisks or by HTML tags) are preserved
    (and turned into HTML tags), unless the markings overlap with 
    predictions, in which case the original is preserved (and still
    turned into an HTML tag if possible)

    Since the model can make "invalid" predictions (mostly those which
    start or end within a LaTeX math mode str), the actual markings
    are not necessarily direct translates from the model's predictions.
    See the helper function `_consolidate_token_preds` for more details
    on how this is implemented.
    
    **Raises**
    Warning messages (`UserWarning`) are printed in the following situations:

    - There are two consecutive tokens within the `pipeline`'s predictions
      of different entity types (e.g. one is predicted to belong within a
      definition and the other within a notation), but the latter token's
      predicted `'entity'` more specifically begins with `'I-'` (i.e. is
      `'I-definition'` or `'I-notation'`) as opposed to `'B-'`.
        - `note`'s name, and path are included in the warning message in
          this case.
    - There are two consecutive tokens within the `pipeline`'s predictions
      which the pipeline predicts to belong to the same entity, and yet
      there is excessive space (specified by `excessive_space_threshold`)
      between the end of the first token and the start of the second.

    """
    mf = MarkdownFile.from_vault_note(note)
    mf.cleanup_formatting()
    # _process_mf(mf)
    first_non_metadata_line, see_also_line = _get_main_text_lines(mf)
     
    main_text = mf.text_of_lines(first_non_metadata_line, see_also_line)
    main_text = _format_main_text_and_add_html_tag_data(
        note, pipeline, add_boxing_attr_to_existing_def_and_notat_markings,
        excessive_space_threshold, main_text)
    _write_text_with_html_tag_preds_to_note(
        note, mf, main_text, first_non_metadata_line, see_also_line)



In the following examples, we mock pipeline objects instead of using actual ones.

In the below example, we run the `auto_mark_def_and_notats` function on a note that has double asterisks `**` surrounding parts of the text that introduced definitions or notations. In these cases, appropriate HTML tags replace the double asterisks instead.

In [ ]:
with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir,
      mock.patch('__main__.pipelines.token_classification.TokenClassificationPipeline') as mock_pipeline):
    temp_vault = Path(temp_dir) / 'test_vault_6'
    shutil.copytree(_test_directory() / 'test_vault_6', temp_vault)

    mock_pipeline.tokenizer.model_max_length = 512

    vn = VaultNote(temp_vault, name='reference_with_tag_labels_Definition 2')
    print("Text before:\n\n")
    print(vn.text())
    print("\n\n\nText after:\n")
    auto_mark_def_and_notats(vn, mock_pipeline)
    print(vn.text())
    mf = MarkdownFile.from_vault_note(vn)
    assert mf.has_tag('_auto/def_and_notat_identified')




Text before:


---
cssclass: clean-embeds
aliases: []
tags: [_meta/literature_note, _meta/definition, _meta/notation]
---
# Ring of integers modulo $n$[^1]

Let $n \geq 1$ be an integer. The **ring of integers modulo $n$**, denoted by **$\mathbb{Z}/n\mathbb{Z}$**, is, informally, the ring whose elements are represented by the integers with the understanding that $0$ and $n$ are equal.

More precisely, $\mathbb{Z}/n\mathbb{Z}$ has the elements $0,1,\ldots,n-1$.

...


# See Also
- [[reference_with_tag_labels_Exercise 1|reference_with_tag_labels_Z_nZ_is_a_ring]]
# Meta
## References

## Citations and Footnotes
[^1]: Kim, Definition 2



Text after:

---
cssclass: clean-embeds
aliases: []
tags: [_meta/definition, _meta/literature_note, _auto/def_and_notat_identified, _meta/notation]
---
# Ring of integers modulo $n$[^1]

Let $n \geq 1$ be an integer. The <b definition="" style="border-width:1px;border-style:solid;padding:3px">ring of integers modulo $n$</b>, denoted by <span notation="" s

In [ ]:
# TODO: more examples with pipeline mocking actual outputs

### Miscellaneous marking formatting

The following is helper code for making predictions on the author's writing

In [ ]:
#| export

def latex_highlight_formatter(text: str, pred: HTMLTagWithIndices) -> str:
    r"""
    Formats definitions and notations with LaTeX highlighting commands.
    
    Rules:
    - Definitions: \hldef{text}
    - Notations:
        - $...$ -> \hl{$...$}
        - $$...$$ -> $$\hlin{...}$$
        - \begin{align*}...\end{align*} -> \hlalign{\begin{align*}...\end{align*}}
    """
    is_definition = 'definition' in pred.tag.attrs
    
    # --- Case 1: Definition ---
    if is_definition:
        return f"\\hldef{{{text}}}"
    
    # --- Case 2: Notation ---
    # We need to detect the type of math environment in `text`
    
    stripped_text = text.strip()
    
    # Subcase 2a: Display Math with $$...$$
    if stripped_text.startswith('$$') and stripped_text.endswith('$$'):
        # Extract content inside $$...$$
        # Content length is len(text) - 4 (for the two $$ pairs)
        # We need to handle potential whitespace, so we use indices based on the strip
        inner_content = stripped_text[2:-2] 
        return f"$$\\hlin{{{inner_content}}}$$"
    
    # Subcase 2b: Align Environment
    # Checks for \begin{align*} or \begin{align}
    if r'\begin{align' in text:
        return f"\\hlalign{{{text}}}"
    
    # Subcase 2c: Inline Math $...$ (Default for notations)
    # Note: We wrap the *entire* text (including $) in \hl{...}
    # User requirement: "if $blah$, then \hl{$blah$}"
    return f"\\hl{{{text}}}"


In [ ]:
#| notest
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
# from trouver.machine_learning.tokenize.def_and_notat_token_classification import (
#     predict_and_mark_def_and_notats,
#     latex_highlight_formatter
# )
from types import SimpleNamespace

# 1. Load the Model
# Replace 'hyunjongkim/math-def-and-notat-token-classification' with your actual model path/ID
# We use 'token-classification' task. 'aggregation_strategy="simple"' helps merge B- and I- tags automatically.
model = AutoModelForTokenClassification.from_pretrained('hyunjongkimmath/def_and_notat_token_classification_model_modernbert_base')
tokenizer = AutoTokenizer.from_pretrained('hyunjongkimmath/def_and_notat_token_classification_model_modernbert_base')
def_notat_classifier = pipeline('ner', model=model, tokenizer=tokenizer)

# 2. Setup the Inputs
# Text with definition ("Galois group"), inline notation ($...$), and display notation ($$...$$)
text = (
    r"Let $L/K$ be a separable, normal algebraic field extension. The Galois group $\operatorname{Gal}(L/K)$ is defined as the automorphism group."
    "\n\n"
    r"Consider the sequence: $$ 0 \to A \to B \to C \to 0 $$"
)

# A dummy note object (required by the function for logging)
# note = SimpleNamespace(name="Example Note", path="Example.md")

# 3. Run Prediction and Formatting
formatted_text = predict_and_mark_def_and_notats(
    main_text=text,
    pipeline=def_notat_classifier,
    # note=note,
    formatter=latex_highlight_formatter,
    excessive_space_threshold=2
)

# 4. View Result
print(formatted_text)

# Expected Output (conceptual):
# The \hldef{Galois group} \hl{$\operatorname{Gal}(L/K)$} is defined by the kernel. 
# Consider the sequence: $$\hlin{ 0 \to A \to B \to C \to 0 }$$


Device set to use cpu
Compiling the model with `torch.compile` and using a `torch.cpu` device is not supported. Falling back to non-compiled mode.


Let $L/K$ be a separable, normal algebraic field extension. The \hldef{Galois group} \hl{$\operatorname{Gal}(L/K)$} is defined as the automorphism group.

Consider the sequence: $$ 0 \to A \to B \to C \to 0 $$


In [ ]:
#| hide
from bs4 import BeautifulSoup

# --- Setup ---
soup = BeautifulSoup("", 'html.parser')

def make_pred(tag_type, text):
    # Helper to create a dummy prediction
    attrs = {tag_type: ""}
    tag = soup.new_tag("span", **attrs)
    # Indices don't matter for the formatter, only the text passed to it
    return HTMLTagWithIndices(tag, 0, 0)

# --- Test 1: Definition ---
text_def = "Galois group"
pred_def = make_pred("definition", text_def)
assert latex_highlight_formatter(text_def, pred_def) == r"\hldef{Galois group}"

# --- Test 2: Inline Notation ---
text_inline = "$x$"
pred_not = make_pred("notation", text_inline)
assert latex_highlight_formatter(text_inline, pred_not) == r"\hl{$x$}"

# --- Test 3: Display Notation ($$) ---
text_display = "$$x^2$$"
pred_not = make_pred("notation", text_display)
# Expect: $$\hlin{x^2}$$
assert latex_highlight_formatter(text_display, pred_not) == r"$$\hlin{x^2}$$"

# Test 3b: Display with whitespace
text_display_ws = "$$  y^2  $$"
# Expect: $$\hlin{  y^2  }$$  (Inner content preserved)
assert latex_highlight_formatter(text_display_ws, pred_not) == r"$$\hlin{  y^2  }$$"

# --- Test 4: Align Notation ---
text_align = r"""\begin{align*}
x &= y \\
y &= z
\end{align*}"""
pred_not = make_pred("notation", text_align)

expected_align = r"\hlalign{" + text_align + "}"
assert latex_highlight_formatter(text_align, pred_not) == expected_align

print("All latex_highlight_formatter tests passed!")


All latex_highlight_formatter tests passed!


## Using an agent to verify

The NER model approach above is quite fast and often accurate, even on a personal use computer and CPU. Nevertheless, it does make mistakes, so manual correction is needed. The following is an agentic process that attempts to find if a the NER model has made mistakes when marking definitions and notations on a note and thus the note needs manual correction.

In [ ]:
#| export
DEF_NOTAT_VERIFY_SYSTEM_PROMPT = r"""
You are an expert auditor of mathematical texts. Your task is to validate semantic HTML markings (attributes: "definition" or "notation") within an excerpt. You must determine if the current markings correctly identify **newly introduced** terms while ignoring **contextual** objects.

### I. The Core Binary Classification
For every mathematical object or term, apply this binary test.

**1. THE TARGET (Must be Marked)**
A term is a Target if and only if the excerpt **establishes its meaning for the first time** intended for use beyond the immediate sentence.
*   **New Constructions:** "For any field $K$, let $G_K$ denote..." (Mark $G_K$).
*   **Formal Definitions:** "We say a sheaf is <b definition>flasque</b> if..." (Mark "flasque").
*   **Explicit Assignments:** "We define the L-series $L(s, \chi)$ as..." (Mark "L-series" and $L(s, \chi)$).
*   **Self-Contained "Recalls":** If the text says "Recall that X is called Y if [Definition]", and the excerpt contains the actual definition criteria, Mark Y. The author is establishing the definition for this text.
    *   *Correct:* "Recall that $X$ is <b definition>totally disconnected</b> if connected components are points."
    *   *Incorrect (Don't mark):* "Recall the properties of totally disconnected spaces."

**2. THE CONTEXT (Must NOT be Marked)**
Everything else is Context. This includes:
*   **Generic Variables:** "Let $X$ be a scheme..." ($X$ is a generic placeholder).
*   **Specific Instances/Applications:** "Let $G = \text{Gal}(L/K)$..." ($G$ is just a shorthand for this specific proof).
*   **Reminders/Recalls:** "Recall that $H^i$ denotes cohomology..." (The definition exists outside this excerpt).
*   **Process Variables:** "By induction on $n$...", "It suffices to treat...", "Let $f: X \to Y$ be a morphism..."

### II. Auditing Rules

1.  **The "Defined Here" Rule:** Only mark objects if the text explicitly links the symbol/term to its formal name or construction *in this specific excerpt*. If the text implies the reader should already know it (e.g., "Consider the trace defined above"), do not mark it.
2.  **Construction vs. Instance:**
    *   "Let $f$ be the map defined in Eq 1" $\to$ **Context** (Reference to past).
    *   "Let $f$ denote the map..." $\to$ **Target** (Establishing new notation).
3.  **OCR Robustness:** Treat malformed LaTeX (e.g., missing `$`) as valid text. If `L(s)` is a Target but lacks delimiters, it still requires a mark.

### III. Reference Examples

*   **Correct Definition:** "We call a functor $F$ <b definition>representable</b> if..."
    *   *Reason:* Defines the property "representable". $F$ is Context.
*   **Correct Notation:** "Let $\mathbb{A}^n$ denote affine space."
    *   *Reason:* Explicit assignment of global notation.
*   **False Positive (Do not mark):** "Let $C$ be a category. If $C$ has limits..."
    *   *Reason:* $C$ is a generic variable used to set the stage.
*   **False Positive (Do not mark):** "The fiber product $X \times_Y Z$ (see Chapter 1)..."
    *   *Reason:* Reference to a prior definition.

### IV. Output format
Analyze the excerpt based on these rules. Determine `has_incorrect_markings` (Context marked as Target) and `has_missing_markings` (Target not marked). Return the result as a JSON object matching the requested schema. Provide detailed reasoning distinguishing "Target" from "Context".
"""


In [ ]:
#| export
# DEF_NOTAT_VERIFY_SYSTEM_PROMPT = r"""
# You are an expert auditor of mathematical texts, trained to identify errors in semantic markings of definitions and notations introduced in small parts ("excerpts") of mathematical text. Your analysis must be guided by the following principle and rules. The markings are by HTML tags that contain the attribute "definition" or "notation".

# I. The Core Principle: Definitional Intent

# The primary goal of the markings is to determine the EXACT location that a definition or notation in an excerpt is defined at. In principle, a reader with knowledge of all definitions and notations across all excerpts should be able to understand any part of the overarching text. YOUR primary goal is to determine whether or not the markings in the provided excerpt succeed at this goal.

# You must be able to distinguish between:

#     Definitions and notations that are INTRODUCED in the excerpt. This MUST be marked.

#     "Supporting Cast": Local Variables that should not have any bearing outside the immediate excerpt, ALONG WITH pre-existing or standard definitions and notations that are neither introduced nor defined in the excerept.

# A marking is an error if and only if it violates this principle.
# II. The Rules of Auditing

#     Before all else, read the excerpt and identify only Global Main Subjects: objects or notations intended to persist across multiple sections as standard terminology for the entire work. If a variable is introduced to state a specific Theorem or Lemma (e.g., 'Let G be a group' in the context of Theorem 1), it is Supporting Cast for that specific logic and must not be marked.

#         "In this section, K is a field..." -> K is explicitly described as a variable needed to understand other parts of the section.

#     Classify All Other Objects as Supporting Cast: Any variable or object used in a context of application is Supporting Cast.

#         Clues for Supporting Cast (No Mark): 'It suffices to treat...', 'By induction on...', 'We have...', 'Recall that...', 'Note that...'. These phrases indicate the text is using existing tools to perform a proof.

#         Clues for Main Subject (Mark): 'We define...', 'We shall denote...', 'Let us call...'. These indicate the initial assignment of a name/symbol to a concept."
    
#         "Let C be a category..." -> C is Supporting Cast. The text is using **A** (as opposed to some specific) category, not defining what a category is.

#         "Let g: G -> X be a morphism..." -> g, G, and X are Supporting Cast (again, the morphism is **A MORPHISM** (as opposed to some specific morphism)). They are props for the discussion and/or the definitions/notations being defined.

#     Distinguish Construction from Application: An object is a Main Subject only if the notation itself or the specific name is being assigned for the first time.

#         Example: 'We define the L-series L(s,χ)=...' is a Main Subject.

#         Counter-Example: 'Consider the L-series L(s,χ) defined in Chapter 1' is Supporting Cast.

#         Counter-Example: Gal(L/K) in a theorem is an application of a standard construction, and not the introduction of one unless an introduction or reminder is explicit.

#     Construction vs. Instance: An object is a Main Subject only if the text is defining a general construction or global notation.

#         Example: 'For any field K, let G_K​ denote its Galois group.' → This is an introduction of notation G_K​. (Mark it).

#         Counter-Example: 'Let G=Gal(L/K) be the Galois group of the extension.' → This is merely naming a specific instance for the sake of a proof. G is Supporting Cast. (Do not mark)."

#     Distinguishing Definition from Reminder: 
#         Since you only see an excerpt, use the "First Invocation" Heuristic: 
#         Assume a notation/term is being introduced if it meets two criteria:
#         1. It is a Specialized Notation (multicharacter symbols like tr.deg, cd_p, or functional operators) rather than a simple variable (x, y, K).
#         2. The text explicitly links the symbol to its formal name (e.g., "where tr.deg denotes the transcendental degree"). 
        
#         Even if this appears in a "where" clause or a proof, treat it as a Main Subject "in effect" for this excerpt.

#     Handle Malformed Text (OCR Errors): Treat mathematical notation that is missing its $ delimiters (e.g., L\left(X\right)) as if it were properly wrapped. If it represents a Main Subject, it requires a mark.

#     However, you are NOT trying to judge whether the given excerpt is well formatted or even coherent. You are trying to only decide whether or not the excerpt introduces definitions and notations and whether it marks those newly introduced definitions and notations

# III. Illustrative Guide & Edge Cases

# Reference these examples to resolve ambiguity.

#     Example A: Clear Definition

#         Text: "We say that a sheaf F is <b style="border-width:1px;border-style:solid;padding:3px" definition="">flasque</b> if for every open set U, the map... is surjective."

#         Analysis: The Main Subject is the property "flasque." The mark is CORRECT. F and U are Supporting Cast.

#     Example B: The Ambiguous "Let..." Clause

#         Scenario 1 (Supporting Cast):

#             Text: "Let $C$ be a category... We say the inductive limit of $G$ is <b style="border-width:1px;border-style:solid;padding:3px" definition="">universal</b> if..."

#             Analysis: The Main Subject is the property "universal." The object $C$ is merely Supporting Cast, used to provide a context for the definition. Tagging $C$ would be an INCORRECT MARKING. On the other hand Tagging "universal" is a CORRECT MARKING.

#         Scenario 2 (Supporting Cast): In a Proof or Theorem, 'Let X be...' is almost always Supporting Cast. Rule of Thumb: If an object is defined as a specific instance to prove a point (e.g., 'There exists a subextension L′...'), it is Supporting Cast. A Main Subject is typically introduced in a formal 'Definition' block or via the phrase 'We shall call such a [concept] a...'

#     Example C: New Construction

#         Text: "Let $g: G \to X$ and $m: Y \to X$ be morphisms. We define the fiber product, denoted <span style="border-width:1px;border-style:solid;padding:3px" notation="">$G \times_X Y$</span>, as the pullback..."

#         Analysis: g, G, X, m, Y are all Supporting Cast. The Main Subjects are the term "fiber product" and its notation $G \times_X Y$. Both require marks. Tagging g or m would be INCORRECT.

# IV. Final Instructions

#     Analyze: Read the user's text and apply the Core Principle and Rules to identify the definitions and notations that are being introduced and to identify the Supporting Cast.

#     Reason: In your reasoning, explicitly use the terms "Definitions and Notations Introduced" and "Supporting Cast" to justify your findings. State why an object fits into one category or the other.

#     Conclude: Based on your analysis, determine the values for has_incorrect_markings and has_missing_markings. An incorrect marking is a tag on a Supporting Cast member. A missing marking is a lack of a tag on a Main Subject.

#     Format: Provide your result as a valid JSON object matching the requested schema. Your reasoning must be detailed with at least 4 sentences and follow the steps above.

# """

In [ ]:
#| export
# DEF_NOTAT_VERIFY_SYSTEM_PROMPT = r"""
# You are a rigorous Mathematical Text Auditor. Your goal is to identify semantic marking errors in LaTeX/HTML definitions.

# ### 1. RECOGNITION RULES (Crucial)
# - DEFINED TERMS: If the text says "An object X is [Term] if..." or "We call Y a [Term]...", the [Term] IS the primary subject and MUST be tagged.
# - INSTANTIATED OBJECTS: If a specific object is introduced with "Let X be...", "Define X as...", or "Consider the object X...", then X ITSELF is a notation that MUST be tagged.
#     - Example: "Let $\mathbb{A}^m_0$ be the affine space..." -> $\mathbb{A}^m_0$ IS a notation/definition here.
#     - Contrast: "Let $n$ be an integer..." -> $n$ is a generic variable (do not tag).
# - NOTATION: Symbols like $X_{proet}$, $\Delta_f$, or specific spaces like $\mathbb{A}_0^m$ are formal notations. If they are tagged with <b> or <span>, they are CORRECT.
# - UNWRAPPED MATH: If a string looks like math (e.g., has `\left(`, `\mathscr`, `_0`, `^2`), TREAT IT AS MATH NOTATION, even if it lacks dollar sign $ wrappers.
#     - Example: `L\left(X, t\right)` -> Treat as `$L(X, t)$` -> **MUST BE TAGGED** if it's the term being defined.
# - **EXPLICIT DEFINITIONS**: Any symbol followed by "defined... by", ":=", or "given by" is a DEFINED TERM.
#     - Text: "The L-series L(X,t)... defined... by..." -> Both "L-series" AND "L(X,t)" MUST be tagged.

# ### 2. EVALUATION CRITERIA
# - False Positives (has_incorrect_markings): Tags on common english words (e.g., "sine"), repeated tags for terms defined previously, or tags on *truly* generic variables like $i, j, n$ used only as indices.
# - False Negatives (has_missing_markings):
#     - Missing tags on the primary term being introduced (e.g., "The **Swan Conductor**").
#     - Missing tags on the specific mathematical object being instantiated (e.g., "Let **$X$** be...").
#     - Missing tags on new notation symbols (e.g., $:=$, $\mapsto$).
# - **False Negatives (has_missing_markings):**
#     - Missing tags on definitions (even if English term is tagged, the associated notation MUST also be tagged).
#     - **CRITICAL:** Missing tags on "naked" LaTeX code that represents the defined term (e.g., `L\left(...\right)`).

# ### 3. MANDATORY AUDIT STEPS
# 1. List what is actually inside <b> and <span> tags.
# 2. Check if untagged math symbols are *specific objects* (mark missing) or *generic indices* (ignore).
# 3. Only vote TRUE if there is a clear violation.

# ### INSTRUCTIONS:
# 1. Perform a step-by-step audit of the text.
# 2. Explicitly distinguish between "Generic Variables" (n, i, k) and "Specific Instantiated Objects" (X, G, $\mathbb{A}^n$).
# 3. Output valid JSON.

# You are running in 'Structured Output' mode. 
# You must output a VALID JSON object matching the requested schema.

# REASONING MUST BE DETAILED: 4+ sentences minimum.

# """

DEF_NOTAT_VERIFY_USER_PROMPT = r"""
Audit the following excerpt of mathematical text for definition and notation marking errors. Return the result in the specified JSON format.

"""


In [ ]:
#| export
class AuditResult(BaseModel):
    reasoning: str
    has_incorrect_markings: bool
    has_missing_markings: bool

class AuditVoteResult(TypedDict, total=True):
    should_remove: bool
    should_add: bool
    incorrect_tally: int  # Added tally for incorrect markings
    missing_tally: int    # Added tally for missing markings
    error_tally: int    # New: specifically tracks failed LLM calls
    total_votes: int      # Useful for context/percentage
    stats: str

In [ ]:
#| export
INITIAL_ERROR_MESSAGE = "ERROR during LLM call"
def run_strict_audit(
        client: OpenAI,
        text_to_check: str,
        system_prompt: str = DEF_NOTAT_VERIFY_SYSTEM_PROMPT,
        user_prompt: str = DEF_NOTAT_VERIFY_USER_PROMPT,
        temperature: float = 0.7,
        verbose: bool = False,
        ) -> AuditResult:
    """
    Make a client (running on an LLM) audit the definition and notation markings
    in `text_to_check` to see if any definitions/notations should have been marked
    but were not (false negatives) or were marked but should not have been
    (false positives).

    Sends text to the model and forces a JSON response matching AuditResult.
    """
    try:
        response = client.chat.completions.create(
            model="local-model-name",  # Name often ignored by local servers, but required
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"{user_prompt}:\n\n{text_to_check}"},
            ],
            temperature=temperature,  # Low temperature = more deterministic structure
            max_tokens=512,
            # --- THE FIX: NEW STRUCTURED OUTPUT FORMAT ---
            # response_format={
            #     "type": "json_schema",
            #     "json_schema": {
            #         "name": "audit_result", # Identifying name for the schema
            #         "schema": AuditResult.model_json_schema(), # Auto-generates the schema
            #         "strict": True # Forces the model to adhere exactly
            #     }
            # }
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": "audit_result",
                    "strict": True, # Setting this False may allow the model to be longer with its reasoning at the cost of potentially breaking json 
                    # "strict": False,
                    "schema": {
                        "type": "object",
                        "properties": {
                            "reasoning": {"type": "string"},
                            "has_incorrect_markings": {"type": "boolean"},
                            "has_missing_markings": {"type": "boolean"}
                        },
                        "required": ["reasoning", "has_incorrect_markings", "has_missing_markings"],
                        "additionalProperties": False
                    }
                }
            }
        )

        # 1. Get the raw string (The server GUARANTEES this is valid JSON matching schema)
        raw_content = response.choices[0].message.content
        
        # 2. Parse directly into Pydantic
        result = AuditResult.model_validate_json(raw_content)
        
        return result

    except Exception as e:
        if verbose:
            print(f"Audit Failed: {e}")
        return AuditResult(
            reasoning=f"{INITIAL_ERROR_MESSAGE}: {str(e)}",
            has_incorrect_markings=False,
            has_missing_markings=False
        )


In [ ]:
#| export
ALL_AUDITS_FAILED_STRING = "All audit attempts failed due to system errors."
def run_audit_voting(
        client: OpenAI,
        text_to_check: str,
        iterations: int = 3,
        guarantee_success_count: bool = False, # New optional argument
        max_attempts: int = 10,                 # Safety break for the guaranteed mode
        verbose: bool = True,
        system_prompt: str = DEF_NOTAT_VERIFY_SYSTEM_PROMPT,
        user_prompt: str = DEF_NOTAT_VERIFY_USER_PROMPT,
        temperature: float = 0.7,
        ) -> AuditVoteResult:
    """
    Run `run_strict_audit` multiple times to let the LLM within `client`     
    vote against itself concerning whether a text has 
    false negatives or false positives for the marked definitions and notations
    """
    audits: list[AuditResult] = []
    total_attempts = 0
    successful_count = 0

    while True:
        audit = run_strict_audit(
            client, text_to_check, system_prompt, user_prompt, temperature, verbose)
        total_attempts += 1

        # Check if this specific audit was a success (didn't return the error message)
        is_error = audit.reasoning.startswith(INITIAL_ERROR_MESSAGE)
        if not is_error:
            successful_count += 1

        audits.append(audit)

        if verbose:
            status = "Error" if is_error else "Success"
            print(f"Attempt {total_attempts} ({status})")
            print(audit, '\n\n')
        # if verbose:
        #     print(f'Vote {i}')
        #     print(audit, '\n\n')
        # Termination conditions
        if guarantee_success_count:
            # Mode 2: Stop when we have enough successes or hit the safety wall
            if successful_count >= iterations or total_attempts >= max_attempts:
                break
        else:
            # Mode 1: Stop when we hit the fixed iteration count
            if total_attempts >= iterations:

                break
    # Filtering and Tallying
    failures = [a for a in audits if a.reasoning.startswith(INITIAL_ERROR_MESSAGE)]
    successes = [a for a in audits if not a.reasoning.startswith(INITIAL_ERROR_MESSAGE)]
    error_count = len(failures)
    total = len(audits)

    # We base the voting threshold only on successful audits
    # If using Mode 2, this will be exactly `iterations`.
    # If using Mode 1, this will be `iterations - error_count`.
    valid_vote_count = len(successes)
    
    if valid_vote_count == 0:
        return AuditVoteResult(
            should_remove=False, 
            should_add=False, 
            incorrect_tally=0,
            missing_tally=0,
            error_tally=error_count,
            total_votes=total,
            stats=ALL_AUDITS_FAILED_STRING
        )

    threshold = (valid_vote_count // 2) + 1
    inc_votes = sum(1 for audit in successes if audit.has_incorrect_markings)
    mis_votes = sum(1 for audit in successes if audit.has_missing_markings)

    reasonings = [f"[{i+1}] {audit.reasoning}" for i, audit in enumerate(audits)]

    return AuditVoteResult(
        should_remove=inc_votes >= threshold,
        should_add=mis_votes >= threshold,
        incorrect_tally=inc_votes,
        missing_tally=mis_votes,
        error_tally=error_count,
        total_votes=total,
        stats=(
            f"Summary: {inc_votes} inc, {mis_votes} mis, {error_count} failed out of {total} total attempts.\n"
            f"Threshold for action (based on {valid_vote_count} successes): {threshold}\n\n"
            f"Reasonings:\n" + "\n\n".join(reasonings)
        )
    )
    
    # error_count = len(failures)
    # total = len(audits)
    # total = len(audits)
    # # failures = [
    #     # a for a in audits if a.reasoning.startswith(INITIAL_ERROR_MESSAGE)]
    # error_count = len(failures)
    # threshold = (len(audits) // 2) + 1
    # inc_votes = sum(1 for audit in audits if audit.has_incorrect_markings)
    # mis_votes = sum(1 for audit in audits if audit.has_missing_markings)

    # # Check if all attempts resulted in an error object
    # if all(audit.reasoning.startswith(INITIAL_ERROR_MESSAGE) for audit in audits):
    #     return AuditVoteResult(
    #         should_remove=False, 
    #         should_add=False, 
    #         incorrect_tally=0,
    #         missing_tally=0,
    #         error_tally = error_count,
    #         total_votes=total,
    #         stats=ALL_AUDITS_FAILED_STRING
    #     )

    # reasonings = [f"[{i+1}] {audit.reasoning}" for i, audit in enumerate(audits)]
    # # reasonings = [audit.reasoning for audit in audits]

    # return AuditVoteResult(
    #     should_remove=inc_votes >= threshold,
    #     should_add=mis_votes >= threshold,
    #     incorrect_tally=inc_votes,
    #     missing_tally=mis_votes,
    #     error_tally=error_count,
    #     total_votes=total,
    #     stats=(
    #         f"Summary: {inc_votes} inc, {mis_votes} mis, {error_count} failed out of {total}.\n"
    #         f"Threshold for action: {threshold}\n\n"
    #         f"Reasonings:\n" + "\n\n".join(reasonings)
    #     )
    # )
    # # return AuditVoteResult(
    # #     should_remove = inc_votes >= threshold,
    # #     should_add = mis_votes >= threshold,
    # #     incorrect_tally=inc_votes,
    # #     missing_tally=missing_votes,
    # #     total_votes=total,
    # #     stats = f"Inc: {inc_votes}/{len(audits)}, Mis: {mis_votes}/{len(audits)}\n\nReasonings:\n\n{"\n\n".join(reasonings)}"
    # # )

In [ ]:
from unittest.mock import Mock
from openai import OpenAI

# Mock client setup - JSON response for Pydantic parsing
mock_client = Mock()
mock_response = Mock(
    model="deepseek-r1-distill-qwen-7b",
    choices=[Mock(
        message=Mock(
            content='{"reasoning": "Mock reasoning: no issues detected", "has_incorrect_markings": false, "has_missing_markings": false}'
        )
    )]
)
mock_client.chat.completions.create.return_value = mock_response

raw_text_example = r"""
Let $L/K$ be a normal separable algebraic field extension. Its Galois group $\operatorname{Gal}(L/K)$ is defined as ...
"""

# PASS THE MOCK CLIENT
print("Running Audit...")
# with patch('__main__.DEF_NOTAT_VERIFY_SYSTEM_PROMPT', "Mock audit prompt."):
    # ... mock_client setup same as above
result = run_audit_voting(client=mock_client, text_to_check=raw_text_example)


if result:
    print("\nSUCCESS:")
    print(result)
else:
    print("\nFailed to get a result.")


Running Audit...
Vote 0
reasoning='Mock reasoning: no issues detected' has_incorrect_markings=False has_missing_markings=False 


Vote 1
reasoning='Mock reasoning: no issues detected' has_incorrect_markings=False has_missing_markings=False 


Vote 2
reasoning='Mock reasoning: no issues detected' has_incorrect_markings=False has_missing_markings=False 



SUCCESS:
{'should_remove': False, 'should_add': False, 'stats': 'Inc: 0/3, Mis: 0/3\n\nReasonings:\n\nMock reasoning: no issues detected\n\nMock reasoning: no issues detected\n\nMock reasoning: no issues detected'}


In [ ]:
#| notest

if False:
    client = OpenAI(base_url="http://127.0.0.1:1234/v1", api_key="lm-studio")


    raw_text_example = r"""
    In [Kal ], Katz examined a more general class of trigonometric sums, for which he was also able to give similar uniform estimate. To explain this result, let

    $$  h: X \longrightarrow \mathbb{A}_\mathbb{Z}  $$

    be a finitely generated scheme over the affine line $\mathbb{Z}_{\mathbb{Z}}$ above $\mathbb{Z}$. For every prime $p$, every power $q$ of it, and every non-trivial character $\psi$ of $F_{q}$, one defines the following generalized Trigonometric Sum

    $$  \sum_{x \in X\left(\mathbb{F}_{q}\right)} \psi(h(x))  $$

    Let $N$ be the supremum of the dimensions of the fibers of the complexified morphism

    $$  h \otimes \mathbb{C}: X \otimes \mathbb{C} \longrightarrow \mathbb{A}_{\mathbb{C}}  $$
    """

    print("Running Audit...")
    result = run_audit_voting(client, raw_text_example)

    if result:
        print("\nSUCCESS:")
        print(result)
    #     print(f"Reasoning: {result.reasoning}")
    #     print(f"Incorrect Markings: {result.has_incorrect_markings}")
    #     print(f"Missing Markings:   {result.has_missing_markings}")
    else:
        print("\nFailed to get a result.")

In [ ]:

client = OpenAI(base_url="http://127.0.0.1:1234/v1", api_key="lm-studio")

In [ ]:



# raw_text_example = r"""

# 3.4.0. Let $\mathrm{F}$ be an object of $\mathrm{C}$ We denote by $\mathrm{C} / \mathrm{F}$ the following category: The objects of $\mathrm{C} / \mathrm{F}$ are the pairs formed by an object $\mathrm{X}$ of $\mathrm{C}$ and of a morphism $u$ of $\mathrm{X}$ in $\mathrm{F}$. Let $(\mathrm{X}, u)$ and $(\mathrm{Y}, v)$ two objects. A morphism from $(\mathrm{X}, u)$ to $(\mathrm{Y}, v)$ is a $g$ morphism from X to Y such that the following diagram is commutative:

# ![[Pasted image 20250209192841.png]]
# """

# print("Running Audit...")
# result = run_audit_voting(client, raw_text_example)

# if result:
#     print("\nSUCCESS:")
#     print(result)
# #     print(f"Reasoning: {result.reasoning}")
# #     print(f"Incorrect Markings: {result.has_incorrect_markings}")
# #     print(f"Missing Markings:   {result.has_missing_markings}")
# else:
#     print("\nFailed to get a result.")

Running Audit...
Vote 0
reasoning="First, I'll examine each part of the text step by step. The excerpt starts with 'Let F be an object of C' which introduces F as a definition introduced in the current scope. Therefore, F should have a mark. Next, it mentions 'C / F' as the category being defined; this is also a newly introduced notation that needs to be marked. Following that, the objects of C / F are pairs consisting of an object X of C and a morphism u from X to F. Here, both X and u are supporting cast because they're used in the context without being defined within this specific excerpt. The morphisms between these objects are defined as g morphisms satisfying a commutative diagram condition. Since 'g' is introduced specifically for this definition, it should be marked as a new notation. There are no instances of missing marks on main subjects, so that aspect remains unchanged." has_incorrect_markings=False has_missing_markings=True 


Vote 1
reasoning="I will analyze this excerpt